In [1]:
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)
# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)
# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Toggle code</button>''', raw=True)

import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
import pickle, os, gc, re, sys

sns.set();
sns.set_palette("colorblind")

from IPython.display import display, display_html, HTML
from IPython.core.debugger import set_trace
# plt.rcParams["axes.grid"] = False

import Jupyter_module_loader
# sys.path.insert(1, '/path/to/application/app/folder')
from utility import *
# import gaussian_kde

# import warnings
# warnings.filterwarnings('ignore')

import holoviews as hv
from holoviews import opts

encoding = "ISO-8859-1"

# you should clone this git to a subdirectory called 'BES_analysis_code' (in some directory - I call it BES_analysis - doesn't matter though)
# %matplotlib inline



Toggle code

In [2]:
dataset_folder = "C:\\Users\\Marios\\Documents\\GitHub\\Thomas Prosser project\\datasets\\"
output_folder = "C:\\Users\\Marios\\Documents\\GitHub\\Thomas Prosser project\\output\\"

In [480]:
def get_dta_meta(fn):
    reader = pd.io.stata.StataReader( fn )
    header = reader.variable_labels()
    labels = reader.value_labels()
    return header,labels

# pd.read_stata(dataset_folder + "CSES"+os.sep+"module 5"+os.sep+"cses5.dta")
header5,labels5 = get_dta_meta(dataset_folder + "CSES"+os.sep+"module 5"+os.sep+"cses5.dta")
header4,labels4 = get_dta_meta(dataset_folder + "CSES"+os.sep+"module 4"+os.sep+"cses4.dta")
header3,labels3 = get_dta_meta(dataset_folder + "CSES"+os.sep+"module 3"+os.sep+"cses3.dta")
header2,labels2 = get_dta_meta(dataset_folder + "CSES"+os.sep+"module 2"+os.sep+"cses2.dta")
header1,labels1 = get_dta_meta(dataset_folder + "CSES"+os.sep+"module 1"+os.sep+"cses1.dta")
headerINT,labelsINT = get_dta_meta(dataset_folder + "CSES"+os.sep+"integrated 1-4"+os.sep+"cses_imd.dta")

In [3]:
CSES_int = pd.read_stata(dataset_folder + "CSES"+os.sep+"integrated 1-4"+os.sep+"cses_imd.dta")

# THERE ARE NO NULL VALUES!

In [4]:
CSES_int["IMD2006"].value_counts()/CSES_int.shape[0]

# 15% missing

2. SECOND HOUSEHOLD INCOME QUINTILE     0.172113
3. THIRD HOUSEHOLD INCOME QUINTILE      0.170377
1. LOWEST HOUSEHOLD INCOME QUINTILE     0.159668
4. FOURTH HOUSEHOLD INCOME QUINTILE     0.153442
5. HIGHEST HOUSEHOLD INCOME QUINTILE    0.143132
9. MISSING                              0.096256
7. VOLUNTEERED: REFUSED                 0.056289
8. VOLUNTEERED: DON'T KNOW              0.048455
6. [SEE ELECTION STUDY NOTES]           0.000267
Name: IMD2006, dtype: float64

In [53]:
## look up the appropriate weights

## look up party links

## Build the income vs party pos chart!

## Also, check out

In [57]:
def try_to_strip_ques_no(x):
    # format:
#     "97.00 NOT APPLICABLE"
#     "\d+\.\d+? ([A-Z].*$)"
    if not isinstance(x,str):
        return x
    match_exists = re.match("[0-9\.]+ (\[?[A-Z].*$)",x)
    if match_exists:
        return match_exists.groups()[0]
    else:
        return x

In [50]:
num_unique = CSES_int.describe(include='all').loc["unique"]


In [51]:
num_unique[num_unique==1].index

Index(['IMD1001', 'IMD1002_VER', 'IMD1002_DOI', 'IMD1016_3'], dtype='object')

In [52]:
CSES_int.drop(['IMD1001', 'IMD1002_VER', 'IMD1002_DOI', 'IMD1016_3'],axis=1)\
    .applymap(lambda x: try_to_strip_ques_no(x)).stack().value_counts().head(50)

# MISSING
# NOT APPLICABLE(: <explanation>)
# MISSING/ABSTRAINED (DID NOT VOTE)
# VOLUNTEERED: DON'T KNOW( WHERE TO PLACE)
# [SEE ELECTION STUDY NOTES]

MISSING                                                        14405922
NOT APPLICABLE                                                  3162242
NOT APPLICABLE: UNICAMERAL SYSTEM                               2917715
1.0                                                             2450637
0.0                                                             2386651
NOT APPLICABLE: NO UPPER HOUSE ELECTION                         2078388
NOT APPLICABLE: NO ROLE OF PRESIDENT                            1645836
NOT APPLICABLE: NO PRESIDENTIAL ELECTION                        1271085
5                                                               1138066
NOT APPLICABLE: NO SECOND ROUND                                  968848
MISSING/ABSTAINED (DID NOT VOTE)                                 952702
2                                                                939348
3                                                                933840
4                                                               

In [56]:
try_to_strip_ques_no("001. [SEE ELECTION STUDY NOTES]")

'[SEE ELECTION STUDY NOTES]'

In [29]:
x = "2. SECOND HOUSEHOLD INCOME QUINTILE"

match_exists = re.match("[0-9\.]+ ([A-Z].*$)",x)
if match_exists:
    print( match_exists.groups()[0] )

SECOND HOUSEHOLD INCOME QUINTILE


In [ ]:
###

# (1) We have a method of matching the common 'non-answer' strings DONE

# (2) We need to double-check year coverage (pace c.a. prosser, expect each country results to line up with elections)

# (3) Look at repeat respondent coverage - don't expect much overlap, but would be nice to know if there's any

# (4) Start processing one column at a time

# (5) Test set with 
# - party (some measure of)
# - income
# - year/election/country

In [49]:
# (2) - Yes same respondents sometimes repolled up to 5 times (impressive ... given 4 modules?)
# check out which/where?

(CSES_int["IMD1006"]+CSES_int["IMD1008_RES"]).value_counts().value_counts()

1    141365
2     27307
3     11212
4      7677
5      4152
dtype: int64

In [62]:
#     IMD1001      >>>         DATASET
#     constant: "CSES-IMD"
#     IMD1002_VER  >>>         DATASET VERSION
#     constant: "VER2019-OCT-17"
#     IMD1002_DOI  >>>         DIGITAL OBJECT IDENTIFIER
#     constant: "doi:10.7804/cses.imd.2019-10-17"

In [90]:
# IMD1003 >>> ID VARIABLE - ELECTION STUDY (NUMERIC POLITY)
# <Numeric Polity Code + Election Year>. <country name> (<election year)

# it is possible to have *two distinct studies* in the same country and same year (e.g. Germany 2002, mail-back and telephone)
# they 'solve' this by giving numbers other than 0 to *4th* polity code (seems like if there's ever *more than one* it starts 1,2,3... else 0)

# actually, doesn't seem like a consistent rule - e.g. UNITED STATES (2012) has a 2 but no 1
# (CSES_int["IMD1003"][CSES_int["IMD1003"].apply(lambda x: x[3]!="0")]).cat.remove_unused_categories().value_counts()
# 05611999. BELGIUM-FLANDERS (1999)     2179
# 27612002. GERMANY (2002 Telephone)    2000
# 05621999. BELGIUM-WALLONIA (1999)     1960
# 84022012. UNITED STATES (2012)        1929
# 27622002. GERMANY (2002 Mail-Back)    1023

# --> STUDY_UID

# IMD1004      >>>         ID VARIABLE - ELECTION STUDY (ALPHABETIC POLITY)
# <three letter ISO 3166, 'alpha-3' country code><_|diff character for above exceptions><election year>
# (55 countries)

# --> COUNTRY CODE

# IMD1006      >>>         ID COMPONENT - POLITY CSES CODE
# 4 figure code, 60 different polities (5 of them reflecting extra same-year studies)
# IMD1006_UN   >>>         ID COMPONENT - POLITY UN CODE
# (first)3 figure part of the CSES code (55 countries)
# Great Britain (because Northern Ireland not present!)
# IMD1006_NAM  >>>         ID COMPONENT - POLITY NAME
# Country name (55 countries)
# Based on United Nations Statistics Division ... except where they aren't!

# --> COUNTRY NAME

22

In [136]:
# IMD1007      >>>         ID COMPONENT - SAMPLE COMPONENT
# this is a grab bag of all the various ways in which samples from individual studies can actually be divided
# (e.g. one subset did X questions, one volunteered to do some extra questions)
# annoyingly, they *don't* reserve "001. [SEE ELECTION STUDY NOTES]" just to refer to "no sample split"
# but also the first part of the split if there is one (not the end of the world, but still!)
# ((CSES_int["IMD1007"]!="001. [SEE ELECTION STUDY NOTES]").groupby(CSES_int["IMD1003"]).mean()>0).sum()
# 22 election studies have some sort of split

# -> DUMMY

# IMD1008_YEAR >>>         ID COMPONENT - ELECTION YEAR
# 21 years, 1996-2016 (one year per year!) (int16)

# -> YEAR

# IMD1008_MOD_1  >>>       ID COMPONENT - MODULE 1
# IMD1008_MOD_2  >>>       ID COMPONENT - MODULE 2
# IMD1008_MOD_3  >>>       ID COMPONENT - MODULE 3
# IMD1008_MOD_4  >>>       ID COMPONENT - MODULE 4
# All dichotomous categories [00. NOT IN CSES MODULE <no.> < 01. IN CSES MODULE <no.>]
mod = {}
for mod_no in [1,2,3,4]:
    mod[mod_no] = CSES_int["IMD1008_MOD_"+str(mod_no)]=="01. IN CSES MODULE "+str(mod_no)

# technically the modules each have an official non-overlapping time period
# in practice, a small handful of studies in each module overlap adjacent years

# --> CONVENIENCE MASKS

# bi_modual = (CSES_int[search(CSES_int,"IMD1008_MOD").index].applymap(lambda x: not ("NOT" in x)).sum(axis=1).sort_values()==2)
# CSES_int["IMD1003"][bi_modual].cat.remove_unused_categories().value_counts()

# 1303 rows appear in two modules ... everything else 1 and only 1
# 62002002. PORTUGAL (2002)    1303
# (administered during module1->2 transition, so did both questionnaires)

In [ ]:
# IMD1005      >>>         ID VARIABLE - RESPONDENT
# <4 digit numeric polity code><election year><10 digit respondent code>
# Unique row id
# Proposed as means of merging in extra variables from base CSES modules

# --> use for merging!

# IMD1008_RES  >>>         ID COMPONENT - RESPONDENT WITHIN ELECTION STUDY
# 10 digit respondent code within study
# unique per study
# not necessarily unique per country (not clear if same code means same person!)

# --> this doesn't seem particularly necessary unless you think they *are* the same person?
# check their demographics!

In [202]:
# IMD1009      >>>         ELECTION TYPE
# 10. PARLIAMENTARY/LEGISLATIVE                       216621
# 12. PARLIAMENTARY/LEGISLATIVE AND PRESIDENTIAL       46688
# 20. PRESIDENTIAL                                     16683
# 13. PARLIAMENTARY/LEGISLATIVE AND PRIME MINISTER      1091

# 13 is just: 37601996. ISRAEL (1996)    1091
# Wowzer - weird story of the 92-96 Israeli Knesset session, leading to direct elections for prime minister 96-01
# exclude 13 and you've basically just got two binary flags

# IMD1010_1    >>>         ELECTION STUDY WEIGHT: SAMPLE
# IMD1010_2    >>>         ELECTION STUDY WEIGHT: DEMOGRAPHIC
# IMD1010_3    >>>         ELECTION STUDY WEIGHT: POLITICAL

# CSES_int["IMD1010_1"].hist(bins=100)
# CSES_int["IMD1010_2"].hist(bins=100)
# CSES_int["IMD1010_3"].hist(bins=100)
# 62+% == 1.0 (not super detailed weighting)

# IMD1011_M    >>>         DATE 1ST ROUND ELECTION BEGAN - MONTH
# IMD1011_D    >>>         DATE 1ST ROUND ELECTION BEGAN - DAY
# IMD1011_Y    >>>         DATE 1ST ROUND ELECTION BEGAN - YEAR
# month is a category ... but day and year are numbers!

# IMD1012_M    >>>         DATE 2ND ROUND ELECTION BEGAN - MONTH
# IMD1012_D    >>>         DATE 2ND ROUND ELECTION BEGAN - DAY
# IMD1012_Y    >>>         DATE 2ND ROUND ELECTION BEGAN - YEAR
# same, but extra "NOT APPLICABLE: NO SECOND ROUND"

# IMD1013_M    >>>         DATE QUESTIONNAIRE ADMINISTERED - MONTH
# IMD1013_D    >>>         DATE QUESTIONNAIRE ADMINISTERED - DAY
# IMD1013_Y    >>>         DATE QUESTIONNAIRE ADMINISTERED - YEAR
# same but with a "MISSING" category


# IMD1014_1    >>>         INTERVIEW TIMING - NUMBER OF DAYS BETWEEN
#                          INTERVIEW AND FIRST ROUND OF ELECTION

# number (0-362) and "9995. NOT ASCERTAINED,"9999. MISSING"

# IMD1014_2    >>>         INTERVIEW TIMING - NUMBER OF DAYS BETWEEN
#                          INTERVIEW AND SECOND ROUND OF ELECTION
# # number (0-362) and "9995. NOT ASCERTAINED,"9999. MISSING","9996. NOT APPLICABLE: NO SECOND ROUND"
# IMD1015      >>>         STUDY CONTEXT (Timing of study relative to election.)
# 1. POST-ELECTION STUDY                     232554
# 2. PRE-ELECTION AND POST-ELECTION STUDY     39909
# 3. BETWEEN ROUNDS                            8620

# Modes of interview used within the respective election study.
# NOT CLEAR TO ME!
# IMD1016_1    >>>         MODE OF INTERVIEW - STUDY FIRST
# 1. FACE-TO-FACE                         197938
# 2. TELEPHONE                             57925
# 3. MAIL OR SELF-COMPLETED SUPPLEMENT     22890
# 4. INTERNET                               2328
# 9. MISSING                                   2
# IMD1016_2    >>>         MODE OF INTERVIEW - STUDY SECOND
# 9. MISSING                              250557
# 3. MAIL OR SELF-COMPLETED SUPPLEMENT     30526

## I don't understand what STUDY SECOND/THIRD refer to - STUDY FIRST has complete (bar 2 missing) coverage
# Ah - I get it - multi-part studies with a second mail-in component 
# (below is the IMD1016_1 breakdown for IMD1016_2=="3. MAIL OR SELF-COMPLETED SUPPLEMENT")
# CSES_int["IMD1016_1"][CSES_int["IMD1016_2"]=="3. MAIL OR SELF-COMPLETED SUPPLEMENT"].cat.remove_unused_categories().value_counts()
# 2. TELEPHONE       18758
# 1. FACE-TO-FACE    11768

# IMD1016_3    >>>         MODE OF INTERVIEW - STUDY THIRD
# 9. MISSING    281083


In [199]:
CSES_int["IMD1016_1"].value_counts().sum()

281083

In [200]:
CSES_int.shape

(281083, 276)

In [151]:
CSES_int["IMD1003"][CSES_int["IMD1009"]=="20. PRESIDENTIAL"].cat.remove_unused_categories().value_counts()

Series([], Name: IMD1003, dtype: int64)

In [145]:
CSES_int["IMD1008_RES"].groupby(CSES_int["IMD1006_UN"]).apply(lambda x: x.value_counts().head(1)).sort_values()

IMD1006_UN            
008         0000001613    1
804         0000000344    1
792         0000003084    1
688         0000016556    1
499         0000000344    1
440         0000000344    1
417         0000000344    1
404         0000000344    1
380         0000146104    1
372         0011830900    1
348         0000000344    1
233         0000000344    1
858         0000000344    1
191         0000000344    1
032         0000001396    1
158         0000802353    1
040         0000000344    1
100         0006114112    1
208         0000000099    2
826         0000016408    2
036         0000001286    2
752         0000201586    2
710         0000000344    2
703         0000001135    2
056         0000001256    2
428         0000000344    2
528         0000000358    2
246         0000000419    2
376         0000002041    2
112         0000000344    2
152         0000000759    2
250         0000000680    3
764         0000000344    3
756         0000010765    3
300         0000000344   

In [58]:
CSES_int["IMD1006"]

0         0080
1         0080
2         0080
3         0080
4         0080
          ... 
281078    7100
281079    7100
281080    7100
281081    7100
281082    7100
Name: IMD1006, Length: 281083, dtype: object

In [236]:
# Demographics

#     IMD2001_1    >>>         AGE OF RESPONDENT (IN YEARS)

# MISSING mostly here but also distributed throughout at low levels:
# 41702005. KYRGYZSTAN (2005)           2000
# 60402001. PERU (2001)                 1118
# 60402000. PERU (2000)                 1102
# "9998. DON'T KNOW" exist, but not very high
# "9997. REFUSED" bit higher
# Age runs from 15 to 115
# Yes, a bunch of these are younger than required to vote (sometimes intentional as part of rep. study ... mostly not)

#     IMD2001_2    >>>         AGE OF RESPONDENT (IN CATEGORIES)
# 01. YOUNGEST - 24 YEARS    30711
# 02. 25-34 YEARS            47624
# 03. 35-44 YEARS            53280
# 04. 45-54 YEARS            51267
# 05. 55-64 YEARS            44127
# 06. 65 - OLDEST            49988
# 9997. REFUSED                561
# 9998. DON'T KNOW             118
# 9999. MISSING               3407

# YOUNGEST varies from context to context!
# Whole bunch of studies have as ceiling above which everything is X+ (46+ Peru_2001-2, Norwar_2009 76+, US_2008 90+)

#     IMD2002      >>>         GENDER
# 1. MALE                          133456
# 2. FEMALE                        147175
# 3. [SEE ELECTION STUDY NOTES]         1
# 7. REFUSED                           20
# 9. MISSING                          431

# [SEE ELECTION STUDY NOTES] -> 03.     Transsexual or Transgender (but only New Zealand_2014)

#     IMD2003      >>>         EDUCATION
# 0. NONE (NO EDUCATION) / ILLITERATE               16751
# 1. PRIMARY EDUCATION/LOWER SECONDARY EDUCATION    84810
# 2. HIGHER SECONDARY EDUCATION                     81425
# 3. POST-SECONDARY (NON-UNIVERSITY) EDUCATION      43262
# 4. UNIVERSITY EDUCATION                           50335
# 6. OTHER                                             41
# 7. VOLUNTEERED: REFUSED                             757
# 8. VOLUNTEERED: DON'T KNOW                          432
# 9. MISSING                                         3270

# These are *qualifications complete not just enrolled*
# earlier(1-3) CSES modules split by incomplete/complete
# there is no 5.!

#     IMD2004      >>>         MARITAL STATUS (current - e.g. divorced and living as married => 1)

# 1. MARRIED OR LIVING TOGETHER AS MARRIED                                         169499
# 2. WIDOWED                                                                        20906
# 3. DIVORCED OR SEPARATED (MARRIED BUT SEPARATED/NOT LIVING WITH LEGAL SPOUSE)     18905
# 4. SINGLE, NEVER MARRIED                                                          57385
# 5. [SEE ELECTION STUDY NOTES]                                                        97
# 7. VOLUNTEERED: REFUSED                                                             680
# 8. VOLUNTEERED: DON'T KNOW                                                          214
# 9. MISSING                                                                        13397

# Data missing for CHILE (1999, 2005), JAPAN (1996),| KYRGYZSTAN (2005), SOUTH KOREA (2000), and THAILAND (2001).
# [SEE ELECTION STUDY NOTES] -> directed to look at individual election study notes!

#     IMD2005      >>>         RELIGIOUS DENOMINATION 
# 1. CATHOLIC                                                                                                      89835
# 2. PROTESTANT                                                                                                    40810
# 3. ORTHODOX/EASTERN CATHOLIC CHURCHES                                                                            20786
# 4. OTHER CHRISTIAN (NO DENOMINATION GIVEN; INDEPENDENT; NON-AFFILIATED; NON-TRADITIONAL PROTESTANTS; MORMONS)     7022
# 5. JEWISH                                                                                                         4164
# 6. ISLAM - SUNNI                                                                                                  1132
# 7. ISLAM - OTHER                                                                                                  6012
# 8. BUDDHISM                                                                                                      11661
# 9. HINDUISM                                                                                                        320
# 10. INDIGENOUS                                                                                                    3240
# 11. ETHNORELIGIONS                                                                                                 375
# 12. NON-BELIEVERS                                                                                                42767
# 13. AGNOSTICS                                                                                                      697
# 96. OTHER: NOT SPECIFIED                                                                                          6216
# 97. VOLUNTEERED: REFUSED                                                                                          1704
# 98. VOLUNTEERED: DON'T KNOW                                                                                       1497
# 99. MISSING                                                                                                      42845

# Some fiddly harmonization between modules since they don't use a single system

#     IMD2006      >>>         HOUSEHOLD INCOME

# 1. LOWEST HOUSEHOLD INCOME QUINTILE     44880
# 2. SECOND HOUSEHOLD INCOME QUINTILE     48378
# 3. THIRD HOUSEHOLD INCOME QUINTILE      47890
# 4. FOURTH HOUSEHOLD INCOME QUINTILE     43130
# 5. HIGHEST HOUSEHOLD INCOME QUINTILE    40232
# 6. [SEE ELECTION STUDY NOTES]              75
# 7. VOLUNTEERED: REFUSED                 15822
# 8. VOLUNTEERED: DON'T KNOW              13620
# 9. MISSING                              27056

# Income ranges shown represent sample quintiles (not population quintiles).
# *Where possible* - often studies already only provide "quintile" data so they've got to stick with that
# And data on what the same quintile ranges should be is not perfect
# Missing for BELGIUM (2003), KYRGYZSTAN (2005), LATVIA (2010), SOUTH AFRICA (2009) and SOUTH KOREA (2012). 
# *many* study specific notes!

#     IMD2007      >>>         RURAL OR URBAN RESIDENCE

# 1. RURAL AREA OR VILLAGE            59153
# 2. SMALL OR MIDDLE-SIZED TOWN       54782
# 3. SUBURBS OF LARGE TOWN OR CITY    35391
# 4. LARGE TOWN OR CITY               82436
# 7. VOLUNTEERED: REFUSED               113
# 8. VOLUNTEERED: DON'T KNOW            139
# 9. MISSING                          49069

# Not all studies use their schema, several just categorise based on resident area population size, so check notes
# MISSING for BELIGUM-FLANDERS (1999), BELGIUM-WALLONIA (1999), BRAZIL (2014), CANADA (2008, 2011, 2015), CHILE
#          | (1999, 2005, 2009), GERMANY (2002 Mail-Back), GREAT BRITAIN 
#          | (1997), HONG KONG (1998, 2000, 2004, 2008, 2012,), JAPAN (1996),
#          | KYRGYZSTAN (2005), TAIWAN (2004, 2008, 2012), THAILAND (2001),
#          | and the UNITED STATES (2008, 2012)


CSES_int["IMD2001_1"].cat.remove_unused_categories()

# 9997. REFUSED < 9998. DON'T KNOW < 9999. MISSING
# CSES_int["IMD3001_PR_1"].value_counts().sort_index()


0. RESPONDENT DID NOT CAST A BALLOT                                                     5508
1. RESPONDENT CAST A BALLOT                                                            30720
9999993. VOLUNTEERED: RESPONDENT NOT REGISTERED ON ELECTORAL LISTS [IF APPLICABLE]       249
9999995. NOT APPLICABLE: UNICAMERAL SYSTEM                                            104155
9999996. NOT APPLICABLE: NO UPPER HOUSE ELECTION                                      107892
9999997. VOLUNTEERED: REFUSED                                                             88
9999998. VOLUNTEERED: DON'T KNOW                                                         179
9999999. MISSING                                                                       32292
Name: IMD3001_UH, dtype: int64

In [239]:
# Turnout just measured as "casting a ballot" -> not necessarily valid!
# IMD3001      >>>         TURNOUT - MAIN ELECTION
# 0. RESPONDENT DID NOT CAST A BALLOT                                                    40751
# 1. RESPONDENT CAST A BALLOT                                                           230766
# 9999993. VOLUNTEERED: RESPONDENT NOT REGISTERED ON ELECTORAL LISTS [IF APPLICABLE]       674
# 9999997. VOLUNTEERED: REFUSED                                                           1964
# 9999998. VOLUNTEERED: DON'T KNOW                                                        1495
# 9999999. MISSING                                                                        5433

# IMD3001_PR_1 >>>         TURNOUT - CURRENT PRESIDENTIAL ELECTION - ROUND 1
# 0. RESPONDENT DID NOT CAST A BALLOT                                                     6903
# 1. RESPONDENT CAST A BALLOT                                                            54346
# 9999993. VOLUNTEERED: RESPONDENT NOT REGISTERED ON ELECTORAL LISTS [IF APPLICABLE]       262
# 9999995. NOT APPLICABLE: NO ROLE OF PRESIDENT                                         103216
# 9999996. NOT APPLICABLE: NO PRESIDENTIAL ELECTIONS                                    113405
# 9999997. VOLUNTEERED: REFUSED                                                           1021
# 9999998. VOLUNTEERED: DON'T KNOW                                                         576
# 9999999. MISSING                                                                        1354

# IMD3001_PR_2 >>>         TURNOUT - CURRENT PRESIDENTIAL ELECTION - ROUND 2
# 0. RESPONDENT DID NOT CAST A BALLOT                                                     2255
# 1. RESPONDENT CAST A BALLOT                                                            16246
# 9999993. VOLUNTEERED: RESPONDENT NOT REGISTERED ON ELECTORAL LISTS [IF APPLICABLE]         2
# 9999995. NOT APPLICABLE: NO ROLE OF PRESIDENT                                         104307
# 9999996. NOT APPLICABLE: NO PRESIDENTIAL ELECTIONS / NO SECOND ROUND                  142591
# 9999997. VOLUNTEERED: REFUSED                                                            178
# 9999998. VOLUNTEERED: DON'T KNOW                                                         123
# 9999999. MISSING                                                                       15381

# IMD3001_LH   >>>         TURNOUT - CURRENT LOWER HOUSE ELECTION
# 0. RESPONDENT DID NOT CAST A BALLOT                                                    39256
# 1. RESPONDENT CAST A BALLOT                                                           208122
# 9999993. VOLUNTEERED: RESPONDENT NOT REGISTERED ON ELECTORAL LISTS [IF APPLICABLE]       672
# 9999996. NOT APPLICABLE: NO LOWER HOUSE ELECTION                                       24170
# 9999997. VOLUNTEERED: REFUSED                                                           1645
# 9999998. VOLUNTEERED: DON'T KNOW                                                        1110
# 9999999. MISSING                                                                        6108

# IMD3001_UH   >>>         TURNOUT - CURRENT UPPER HOUSE ELECTION
# 0. RESPONDENT DID NOT CAST A BALLOT                                                     5508
# 1. RESPONDENT CAST A BALLOT                                                            30720
# 9999993. VOLUNTEERED: RESPONDENT NOT REGISTERED ON ELECTORAL LISTS [IF APPLICABLE]       249
# 9999995. NOT APPLICABLE: UNICAMERAL SYSTEM                                            104155
# 9999996. NOT APPLICABLE: NO UPPER HOUSE ELECTION                                      107892
# 9999997. VOLUNTEERED: REFUSED                                                             88
# 9999998. VOLUNTEERED: DON'T KNOW                                                         179
# 9999999. MISSING                                                                       32292



0320001. ARG - Front for the Victory                    630
0320002. ARG - Let's Change                             416
0320003. ARG - United for a New Alternative             100
0320004. ARG - Workers' Left Front                       29
0320005. ARG - Progressives                              16
                                                      ...  
9999995. NOT APPLICABLE: NO ROLE OF PRESIDENT        103216
9999996. NOT APPLICABLE: NO PRESIDENTIAL ELECTION    113405
9999997. VOLUNTEERED: REFUSED                          2097
9999998. VOLUNTEERED: DON'T KNOW                        450
9999999. MISSING/ABSTAINED (DID NOT VOTE)             10310
Name: IMD3002_PR_1, Length: 188, dtype: int64

In [251]:
# IMD3002_PR_1 >>>         CURRENT PRESIDENTIAL ELECTION: VOTE CHOICE - ROUND 1
# 0320001. ARG - Front for the Victory                    630
# 0320002. ARG - Let's Change                             416
# 0320003. ARG - United for a New Alternative             100
# 0320004. ARG - Workers' Left Front                       29
# 0320005. ARG - Progressives                              16
#                                                       ...  
# 9999988. NONE OF THE CANDIDATES/PARTIES                                     115
# 9999989. INDEPENDENT CANDIDATE                                               91
# 9999992. OTHER CANDIDATE/PARTY (NOT FURTHER SPECIFIED)                      226
# 9999993. INVALID/BLANK BALLOT                                              1114
# 9999995. NOT APPLICABLE: NO ROLE OF PRESIDENT                            103216
# 9999996. NOT APPLICABLE: NO PRESIDENTIAL ELECTION                        113405
# 9999997. VOLUNTEERED: REFUSED                                              2097
# 9999998. VOLUNTEERED: DON'T KNOW                                            450
# 9999999. MISSING/ABSTAINED (DID NOT VOTE)                                 10310 

# IMD3002_PR_2 >>>         CURRENT PRESIDENTIAL ELECTION: VOTE CHOICE - ROUND 2
# 0320001. ARG - Front for the Victory                                      660
# 0320002. ARG - Let's Change                                               481
# 0760001. BRA - Workers Party                                             4433
# 0760002. BRA - Brazilian Social Dmeocratic Party                         2639
# 2500001. FRA - Union for a Popular Movement                               577
# 2500002. FRA - Socialist Party                                           1037
# 2500007. FRA - National Front                                              61
# 2500008. FRA - Rally For The Republic                                     662
# 6420001. ROU - Romanian Party of Social Democracy                         271
# 6420020. ROU - Democratic Liberal Party                                   536
# 6420022. ROU - Romanian Democratic Convention                             574
# 6420026. ROU - Justice and Truth Alliance                                 874
# 6420041. ROU - National Alliance PSD+PUR                                  534
# 6420101. ROU - PSD-UNPR-PC Electoral Alliance                             287
# 6420102. ROU - Christian-Liberal Alliance                                 550
# 6420111. ROU - Social Democratic Party - Conservative Party               590
# 6880001. SRB - Serbian Progressive Party                                  586
# 6880002. SRB - Democratic Party                                           239
# 8580001. URY - Broad Front                                                537
# 8580002. URY - National Party                                             302
# 9999993. INVALID/BLANK BALLOT                                             629
# 9999995. NOT APPLICABLE: NO ROLE OF PRESIDENT                          104307
# 9999996. NOT APPLICABLE: NO PRESIDENTIAL ELECTION / NO SECOND ROUND    142591
# 9999997. VOLUNTEERED: REFUSED                                            1109
# 9999998. VOLUNTEERED: DON'T KNOW                                          196
# 9999999. MISSING/ABSTAINED (DID NOT VOTE)                               15821

# IMD3002_LH_PL >>>        CURRENT LOWER HOUSE ELECTION: VOTE CHOICE - PARTY LIST
# 0080001. ALB - Socialist Party                         194
# 0080002. ALB - Democratic Party                        263
# 0080003. ALB - Socialist Movement for Integration       60
# 0080004. ALB - Republican Party                        123
# 0080005. ALB - Social Democratic Party                  57
#                                                 ...
# 9999988. NONE OF THE CANDIDATES/PARTIES                                 256
# 9999989. INDEPENDENT CANDIDATE                                          121
# 9999990. OTHER LEFT WING CANDIDATE/PARTY                                 53
# 9999991. OTHER RIGHT WING CANDIDATE/PARTY                                30
# 9999992. OTHER CANDIDATE/PARTY (NOT FURTHER SPECIFIED)                 1446
# 9999993. INVALID/BLANK BALLOT                                          2991
# 9999995. NOT APPLICABLE: NOT A LIST SYSTEM                            75012
# 9999996. NOT APPLICABLE: NO LOWER HOUSE ELECTION                      18641
# 9999997. VOLUNTEERED: REFUSED                                          6197
# 9999998. VOLUNTEERED: DON'T KNOW                                       5411
# 9999999. MISSING/ABSTAINED (DID NOT VOTE)                             38331

# IMD3002_LH_DC >>>        CURRENT LOWER HOUSE ELECTION: VOTE CHOICE - DISTRICT CANDIDATE
# 6080900. PHL - Voted but did not vote for a lower house candidate        60
# 6420002. ROU - People's Party - Dan Diaconescu                          114
# 6420003. ROU - Democratic Union of Hungarians in Romania                 65
# 6420004. ROU - Greater Romania Party                                      6
# 6420011. ROU - Social Liberal Union                                    1034
# 6420200. ROU - Alliance for a Just Romania                              166
# 7560001. CHE - Swiss People's Party                                     324
# 7560002. CHE - Social Democratic Party                                  624
# 7560003. CHE - Christian Democratic People's Party                      359
# 7560004. CHE - The Liberals                                             545
# 7560005. CHE - Green Party                                              149
# 7560006. CHE - Evangelical People's Party                                 9
# 7560009. CHE - Federal Democratic Union                                   1
# 7560021. CHE - Green Liberal Party                                      153
# 7560025. CHE - Conservative Democratic Party                             46
# 7640001. THA - For Thais Party                                          719
# 7640002. THA - Democrat Party                                           912
# 7640003. THA - Thai Pride Party                                          63
# 7640004. THA - Love Thailand Party                                       10
# 7640005. THA - Thai Nation Development Party                             29
# 7640006. THA - National Development of the Motherland Party              15
# 7640007. THA - Peaceloving Party                                          2
# 7640008. THA - Motherland Party (Phak Matunhum Party)                    19
# 7640009. THA - Power of Choburi Party                                     6
# 7640014. THA - People's Power Party                                     974
# 7640017. THA - Social Action Party                                        2
# 7640018. THA - Royal People Party                                         2
# 7640021. THA - Thai Nation Party                                         62
# 7640024. THA - Neutral Democratic Party                                   6
# 7640060. THA - Thais United National Development Party                   24
# 8260001. GBR - Conservatives                                           1320
# 8260002. GBR - Labour                                                  1779
# 8260003. GBR - Liberal Democrats                                        638
# 8260004. GBR - Scottish National Party                                  168
# 8260005. GBR - Green Party                                               46
# 8260006. GBR - Plaid Cymru                                               19
# 8260007. GBR - United Kingdom Independence Party                        141
# 8400001. USA - Republican Party                                        1737
# 8400002. USA - Democratic Party                                        2358
# 8400004. USA - Reform Party                                              18
# 9999988. NONE OF THE CANDIDATES/PARTIES                                 221
# 9999989. INDEPENDENT CANDIDATE                                          901
# 9999990. OTHER LEFT WING CANDIDATE/PARTY                                 44
# 9999992. OTHER CANDIDATE/PARTY (NOT FURTHER SPECIFIED)                 1109
# 9999993. INVALID/BLANK BALLOT                                           828
# 9999995. NOT APPLICABLE: NO DISTRICT CANDIDATE VOTE                  143365
# 9999996. NOT APPLICABLE: NO LOWER HOUSE ELECTION                      22997
# 9999997. VOLUNTEERED: REFUSED                                          3011
# 9999998. VOLUNTEERED: DON'T KNOW                                       1222
# 9999999. MISSING/ABSTAINED (DID NOT VOTE)                             27083

# IMD3002_UH_PL >>>        CURRENT UPPER HOUSE ELECTION: VOTE CHOICE - PARTY LIST
# 0560001. BEL - New Flemish Alliance                                   47
# 0560002. BEL - Socialist Party (Francophone)                         298
# 0560003. BEL - Christian Democratic & Flemish                        253
# 0560004. BEL - Flemish Liberals and Democrats                        258
# 0560005. BEL - Socialist Party Differently                           275
# 0560010. BEL - Humanist Democratic Centre                            133
# 0560012. BEL - Flemish Block                                         122
# 0560013. BEL - Confederated Ecologists                               107
# 0560018. BEL - Live Differently - Flemish Ecologists                  56
# 0560019. BEL - National Front                                         15
# 0560020. BEL - Worker's Party of Belgium                               2
# 0560023. BEL - Alive                                                   6
# 0560026. BEL - Reformist Movement                                    223
# 3920001. JPN - Liberal Democratic Party                             1244
# 3920002. JPN - Democratic Party of Japan                             930
# 3920003. JPN - New Komeito                                           401
# 3920004. JPN - Japanese Communist Party                              222
# 3920005. JPN - Green Wind                                             12
# 3920006. JPN - Social Democratic Party                               110
# 3920007. JPN - Japan Restoration Party                               142
# 3920010. JPN - Your Party                                            119
# 3920011. JPN - People's Life Party                                    20
# 3920015. JPN - New Party Daichi                                        9
# 3920020. JPN - Greens Japan                                           10
# 4840001. MEX - Institutional Revolutionary Party                     808
# 4840002. MEX - National Action Party                                 434
# 4840003. MEX - Democratic Revolution Party                           409
# 4840004. MEX - Labor Party                                            18
# 4840005. MEX - Ecological Green Party of Mexico                       46
# 4840006. MEX - New Alliance Party                                     34
# 4840007. MEX - Citizen`s Movement                                     23
# 6420003. ROU - Democratic Union of Hungarians in Romania              85
# 6420004. ROU - Greater Romania Party                                  85
# 6420007. ROU - National Peasant and Christian Democratic Party        19
# 6420026. ROU - Justice and Truth Alliance                            631
# 6420041. ROU - National Alliance PSD+PUR                             492
# 8580001. URY - Broad Front                                           512
# 8580002. URY - National Party                                        234
# 8580003. URY - Colorado Party                                        120
# 8580004. URY - Independent Party                                      17
# 8580005. URY - Popular Assembly                                        8
# 9999988. NONE OF THE CANDIDATES/PARTIES                               13
# 9999992. OTHER CANDIDATE/PARTY (NOT FURTHER SPECIFIED)                80
# 9999993. INVALID/BLANK BALLOT                                        366
# 9999995. NOT APPLICABLE: UNICAMERAL SYSTEM                        104155
# 9999996. NOT APPLICABLE: NO UPPER HOUSE ELECTION OR LIST VOTE     152025
# 9999997. VOLUNTEERED: REFUSED                                        636
# 9999998. VOLUNTEERED: DON'T KNOW                                     199
# 9999999. MISSING/ABSTAINED (DID NOT VOTE)                          14620

# IMD3002_UH_DC_1 >>>      CURRENT UPPER HOUSE ELECTION: VOTE CHOICE - DISTRICT CANDIDATE 1
# 6160052. POL - Polish National Party                                                         1
# 6160064. POL - Family-Homeland                                                               2
# 6160065. POL - Patriotic Movement of the Republic of Poland                                  4
# 6160068. POL - Social Democracy of Poland                                                   28
# 6160070. POL - Coalition Of The Alliance Of The Democratic Left - The Union of Labor       397
# 6160074. POL - 'Jednomandatowe Okregi Wyborcze'                                              1
#                                                                         ...
# 9999989. INDEPENDENT CANDIDATE                                                             229
# 9999992. OTHER CANDIDATE/PARTY (NOT FURTHER SPECIFIED)                                     942
# 9999993. INVALID/BLANK BALLOT                                                              769
# 9999995. NOT APPLICABLE: UNICAMERAL SYSTEM                                              104155
# 9999996. NOT APPLICABLE: NO UPPER HOUSE ELECTION OR CANDIDATE VOTE                      122735
# 9999997. VOLUNTEERED: REFUSED                                                              533
# 9999998. VOLUNTEERED: DON'T KNOW                                                          1307
# 9999999. MISSING/ABSTAINED (DID NOT VOTE)                                                25249

# IMD3002_UH_DC_2 >>>      CURRENT UPPER HOUSE ELECTION: VOTE CHOICE - DISTRICT CANDIDATE 2
# 6160003. POL - Polish Peasants' (or People's) Party                                        143
# 6160006. POL - Polish Labour Party                                                           2
# 6160009. POL - Democratic Left Alliance                                                     55
# 6160014. POL - League of Polish Families                                                    79
# 6160015. POL - German Minority                                                               2
# 6160020. POL - Left and Democrats                                                           56
#                                                                 ...
# 9999989. INDEPENDENT CANDIDATE                                                              11
# 9999992. OTHER CANDIDATE/PARTY (NOT FURTHER SPECIFIED)                                     105
# 9999995. NOT APPLICABLE: UNICAMERAL SYSTEM                                              104155
# 9999996. NOT APPLICABLE: NO UPPER HOUSE ELECTION OR CANDIDATE VOTE                      124326
# 9999997. VOLUNTEERED: REFUSED                                                               32
# 9999998. VOLUNTEERED: DON'T KNOW                                                           187
# 9999999. MISSING/ABSTAINED (DID NOT VOTE)                                                48394    

# IMD3002_UH_DC_3 >>>      CURRENT UPPER HOUSE ELECTION: VOTE CHOICE - DISTRICT CANDIDATE 3
# 6160001. POL - Civic Platform                                                              157
# 6160002. POL - Law and Justice                                                             124
# 6160003. POL - Polish Peasants' (or People's) Party                                         51
# 6160006. POL - Polish Labour Party                                                           2
#                                                             ...
# 9999995. NOT APPLICABLE: UNICAMERAL SYSTEM                                              104155
# 9999996. NOT APPLICABLE: NO UPPER HOUSE ELECTION OR CANDIDATE VOTE                      124326
# 9999998. VOLUNTEERED: DON'T KNOW                                                            13
# 9999999. MISSING/ABSTAINED (DID NOT VOTE)                                                51980

# IMD3002_UH_DC_4 >>>      CURRENT UPPER HOUSE ELECTION: VOTE CHOICE - DISTRICT CANDIDATE 4
# 6160001. POL - Civic Platform                                                               68
# 6160002. POL - Law and Justice                                                              45
# 6160003. POL - Polish Peasants' (or People's) Party                                          9
# 6160009. POL - Democratic Left Alliance                                                      7
#                                                             ...
# 9999995. NOT APPLICABLE: UNICAMERAL SYSTEM                                              104155
# 9999996. NOT APPLICABLE: NO UPPER HOUSE ELECTION OR CANDIDATE VOTE                      124326
# 9999998. VOLUNTEERED: DON'T KNOW                                                             2
# 9999999. MISSING/ABSTAINED (DID NOT VOTE)                                                52424





0. DID NOT VOTE FOR THE OUTGOING GOVERNMENT (INCUMBENT)               111468
1. VOTED FOR THE OUTGOING GOVERNMENT (INCUMBENT)                       76024
9999996. NOT ASCERTAINED/INCUMBENT CANDIDATE/PARTY DID NOT CONTEST     34889
9999997. VOLUNTEERED: REFUSED                                           8097
9999998. VOLUNTEERED: DON'T KNOW                                        1963
9999999. MISSING / ABSTAINED (DID NOT VOTE)                            48642
Name: IMD3002_OUTGOV, dtype: int64

In [256]:
## Weirdly formatted 'previous vote' question! Is there a convenient 'previous government' variable?

# IMD3002_OUTGOV  >>>      CURRENT MAIN ELECTION: VOTE CHOICE - OUTGOING GOVERNMENT (INCUMBENT)
# 0. DID NOT VOTE FOR THE OUTGOING GOVERNMENT (INCUMBENT)               111468
# 1. VOTED FOR THE OUTGOING GOVERNMENT (INCUMBENT)                       76024
# 9999996. NOT ASCERTAINED/INCUMBENT CANDIDATE/PARTY DID NOT CONTEST     34889
# 9999997. VOLUNTEERED: REFUSED                                           8097
# 9999998. VOLUNTEERED: DON'T KNOW                                        1963
# 9999999. MISSING / ABSTAINED (DID NOT VOTE)                            48642

# IMD3003_PR_1 >>>         TURNOUT - PREVIOUS PRESIDENTIAL ELECTION - ROUND 1
# 0. RESPONDENT DID NOT CAST A BALLOT                                                    8523
# 1. RESPONDENT CAST A BALLOT                                                           36082
# 9999993. VOLUNTEERED: RESPONDENT NOT REGISTERED ON ELECTORAL LISTS [IF APPLICABLE]     1304
# 9999995. NOT APPLICABLE: NO ROLE OF PRESIDENT                                         74635
# 9999996. NOT APPLICABLE: NO PRESIDENTIAL ELECTIONS                                    88868
# 9999997. VOLUNTEERED: REFUSED                                                           194
# 9999998. VOLUNTEERED: DON'T KNOW                                                        965
# 9999999. MISSING                                                                      70512
# IMD3003_PR_2 >>>         TURNOUT - PREVIOUS PRESIDENTIAL ELECTION - ROUND 2
# 0. RESPONDENT DID NOT CAST A BALLOT                                                      979
# 1. RESPONDENT CAST A BALLOT                                                             7327
# 9999993. VOLUNTEERED: RESPONDENT NOT REGISTERED ON ELECTORAL LISTS [IF APPLICABLE]       234
# 9999995. NOT APPLICABLE: NO ROLE OF PRESIDENT                                          76288
# 9999996. NOT APPLICABLE: NO PRESIDENTIAL ELECTIONS / NO SECOND ROUND                  115646
# 9999997. VOLUNTEERED: REFUSED                                                             74
# 9999998. VOLUNTEERED: DON'T KNOW                                                         242
# 9999999. MISSING                                                                       80293
# IMD3003_LH   >>>         TURNOUT - PREVIOUS LOWER HOUSE ELECTION
# 0. RESPONDENT DID NOT CAST A BALLOT                                                    26281
# 1. RESPONDENT CAST A BALLOT                                                           130526
# 9999993. VOLUNTEERED: RESPONDENT NOT REGISTERED ON ELECTORAL LISTS [IF APPLICABLE]      4319
# 9999996. NOT APPLICABLE: NO LOWER HOUSE ELECTION                                       27930
# 9999997. VOLUNTEERED: REFUSED                                                            964
# 9999998. VOLUNTEERED: DON'T KNOW                                                        3712
# 9999999. MISSING                                                                       87351
# IMD3003_UH   >>>         TURNOUT - PREVIOUS UPPER HOUSE ELECTION   
# 0. RESPONDENT DID NOT CAST A BALLOT                                                     2836
# 1. RESPONDENT CAST A BALLOT                                                            12699
# 9999993. VOLUNTEERED: RESPONDENT NOT REGISTERED ON ELECTORAL LISTS [IF APPLICABLE]       395
# 9999995. NOT APPLICABLE: UNICAMERAL SYSTEM                                             82264
# 9999996. NOT APPLICABLE: NO UPPER HOUSE ELECTION                                      101202
# 9999997. VOLUNTEERED: REFUSED                                                            114
# 9999998. VOLUNTEERED: DON'T KNOW                                                         646
# 9999999. MISSING                                                                       80927



In [276]:
# IMD3004_PR_1 >>>         PREVIOUS PRESIDENTIAL ELECTION: VOTE CHOICE - ROUND 1
# 0320001. ARG - Front for the Victory                   678
# 0320004. ARG - Workers' Left Front                      18
# 0320006. ARG - Federal Commitment                       43
#                                                      ...  
# 9999988. NONE OF THE CANDIDATES/PARTIES                                        143
# 9999989. INDEPENDENT CANDIDATE                                                 870
# 9999992. OTHER CANDIDATE/PARTY (NOT FURTHER SPECIFIED)                         151
# 9999993. INVALID/BLANK BALLOT                                                  883
# 9999995. NOT APPLICABLE: NO ROLE OF PRESIDENT                                74635
# 9999996. NOT APPLICABLE: NO PRESIDENTIAL ELECTION                            80647
# 9999997. VOLUNTEERED: REFUSED                                                 1548
# 9999998. VOLUNTEERED: DON'T KNOW                                              2079
# 9999999. MISSING/ABSTAINED (DID NOT VOTE)                                    81825

# IMD3004_PR_2 >>>         PREVIOUS PRESIDENTIAL ELECTION: VOTE CHOICE - ROUND 2
# 0760001. BRA - Workers Party                                             3378
# 0760002. BRA - Brazilian Social Dmeocratic Party                         1025
# 2500001. FRA - Union for a Popular Movement                              1814
# 2500002. FRA - Socialist Party                                           1492
# 6040001. PER - Popular Force                                              400
# 6040020. PER - Christian People's Party                                   637
# 6040050. PER - Peru Wins                                                 1137
# 6160001. POL - Civic Platform                                             589
# 6160002. POL - Law and Justice                                            621
# 6420020. ROU - Democratic Liberal Party                                   422
# 6420111. ROU - Social Democratic Party - Conservative Party               251
# 9999993. INVALID/BLANK BALLOT                                             490
# 9999995. NOT APPLICABLE: NO ROLE OF PRESIDENT                           76288
# 9999996. NOT APPLICABLE: NO PRESIDENTIAL ELECTION / NO SECOND ROUND    117551
# 9999997. VOLUNTEERED: REFUSED                                             197
# 9999998. VOLUNTEERED: DON'T KNOW                                          474
# 9999999. MISSING/ABSTAINED (DID NOT VOTE)                               74317

# IMD3004_LH_PL >>>        PREVIOUS LOWER HOUSE ELECTION: VOTE CHOICE - PARTY LIST
# 0080001. ALB - Socialist Party                         352
# 0080005. ALB - Social Democratic Party                  15
#                                                      ...  
# 9999988. NONE OF THE CANDIDATES/PARTIES                       82
# 9999989. INDEPENDENT CANDIDATE                               170
# 9999990. OTHER LEFT WING CANDIDATE/PARTY                      72
# 9999991. OTHER RIGHT WING CANDIDATE/PARTY                     32
# 9999992. OTHER CANDIDATE/PARTY (NOT FURTHER SPECIFIED)       797
# 9999993. INVALID/BLANK BALLOT                                936
# 9999995. NOT APPLICABLE: NOT A LIST SYSTEM                 55711
# 9999996. NOT APPLICABLE: NO LOWER HOUSE ELECTION           20123
# 9999997. VOLUNTEERED: REFUSED                               5576
# 9999998. VOLUNTEERED: DON'T KNOW                            8512
# 9999999. MISSING/ABSTAINED (DID NOT VOTE)                 106176

# IMD3004_LH_DC >>>        PREVIOUS LOWER HOUSE ELECTION: VOTE CHOICE - DISTRICT CANDIDATE
# 0080001. ALB - Socialist Party                                            360
# 0080005. ALB - Social Democratic Party                                      5
#                                                                         ...  
# 9999988. NONE OF THE CANDIDATES/PARTIES                       93
# 9999989. INDEPENDENT CANDIDATE                               429
# 9999992. OTHER CANDIDATE/PARTY (NOT FURTHER SPECIFIED)       668
# 9999993. INVALID/BLANK BALLOT                                163
# 9999995. NOT APPLICABLE: NO DISTRICT CANDIDATE VOTE       108415
# 9999996. NOT APPLICABLE: NO LOWER HOUSE ELECTION           26945
# 9999997. VOLUNTEERED: REFUSED                               2330
# 9999998. VOLUNTEERED: DON'T KNOW                            4144
# 9999999. MISSING/ABSTAINED (DID NOT VOTE)                  89164

# IMD3004_UH_PL >>>        PREVIOUS UPPER HOUSE ELECTION: VOTE CHOICE - PARTY LIST
# 6420001. ROU - Romanian Party of Social Democracy                   681
# 6420003. ROU - Democratic Union of Hungarians in Romania             60
# 6420004. ROU - Greater Romania Party                                126
# 6420005. ROU - National Liberal Party                                88
# 6420019. ROU - Democratic Party                                      25
# 6420025. ROU - CDR 2000                                              33
# 8580001. URY - Broad Front                                          472
# 8580002. URY - National Party                                       241
# 8580003. URY - Colorado Party                                       100
# 8580004. URY - Independent Party                                     13
# 8580006. URY - Civic Union                                            4
# 9999992. OTHER CANDIDATE/PARTY (NOT FURTHER SPECIFIED)                5
# 9999993. INVALID/BLANK BALLOT                                        23
# 9999995. NOT APPLICABLE: UNICAMERAL SYSTEM                        82264
# 9999996. NOT APPLICABLE: NO UPPER HOUSE ELECTION OR LIST VOTE    129089
# 9999997. VOLUNTEERED: REFUSED                                       668
# 9999998. VOLUNTEERED: DON'T KNOW                                    235
# 9999999. MISSING/ABSTAINED (DID NOT VOTE)                         66956

# IMD3004_UH_DC_1 >>>      PREVIOUS UPPER HOUSE ELECTION: VOTE CHOICE - DISTRICT CANDIDATE 1
# 0760001. BRA - Workers Party                                             703
# 0760002. BRA - Brazilian Social Dmeocratic Party                         409
#                                                                        ...  
# 9999989. INDEPENDENT CANDIDATE                                            39
# 9999992. OTHER CANDIDATE/PARTY (NOT FURTHER SPECIFIED)                    23
# 9999993. INVALID/BLANK BALLOT                                            389
# 9999995. NOT APPLICABLE: UNICAMERAL SYSTEM                             84489
# 9999996. NOT APPLICABLE: NO UPPER HOUSE ELECTION OR CANDIDATE VOTE    107255
# 9999997. VOLUNTEERED: REFUSED                                            394
# 9999998. VOLUNTEERED: DON'T KNOW                                        3543
# 9999999. MISSING/ABSTAINED (DID NOT VOTE)                              

# IMD3004_UH_DC_2 >>>      PREVIOUS UPPER HOUSE ELECTION: VOTE CHOICE - DISTRICT CANDIDATE 2
# 0760001. BRA - Workers Party                                                              99
# 0760002. BRA - Brazilian Social Dmeocratic Party                                          51
#                                                 ...
# 9999989. INDEPENDENT CANDIDATE                                                             7
# 9999995. NOT APPLICABLE: UNICAMERAL SYSTEM                                             84489
# 9999996. NOT APPLICABLE: NO UPPER HOUSE ELECTION OR CANDIDATE VOTE                    110423
# 9999997. VOLUNTEERED: REFUSED                                                            127
# 9999998. VOLUNTEERED: DON'T KNOW                                                         524
# 9999999. MISSING/ABSTAINED (DID NOT VOTE)                                              84691

# IMD3004_UH_DC_3 >>>      PREVIOUS UPPER HOUSE ELECTION: VOTE CHOICE - DISTRICT CANDIDATE 3
# 6160003. POL - Polish Peasants' (or People's) Party                        7
# 6160007. POL - Labour Union                                                3
# 6160008. POL - Freedom Union                                               6
# 6160009. POL - Democratic Left Alliance                                   22
# 6160029. POL - National Party of Pensioners and Retired                    1
# 6160065. POL - Patriotic Movement of the Republic of Poland                2
# 6160081. POL - Coalition Electoral Solidarity                             16
# 6160086. POL - German Minority Association In Opole Silesia                1
# 9999989. INDEPENDENT CANDIDATE                                             2
# 9999995. NOT APPLICABLE: UNICAMERAL SYSTEM                             84489
# 9999996. NOT APPLICABLE: NO UPPER HOUSE ELECTION OR CANDIDATE VOTE    110423
# 9999998. VOLUNTEERED: DON'T KNOW                                           1
# 9999999. MISSING/ABSTAINED (DID NOT VOTE)                              86110



In [285]:
# IMD3005_1    >>>         PARTY IDENTIFICATION: ARE YOU CLOSE TO ANY POLITICAL PARTY
# 1. YES                        120486
# 5. NO                         139221
# 7. VOLUNTEERED: REFUSED         3114
# 8. VOLUNTEERED: DON'T KNOW      6333
# 9. MISSING                     11929

# IMD3005_2    >>>         PARTY IDENTIFICATION: DO YOU FEEL CLOSER TO ONE PARTY
# 1. YES                         47643
# 5. NO                          77333
# 7. VOLUNTEERED: REFUSED         1574
# 8. VOLUNTEERED: DON'T KNOW      4906
# 9. MISSING                    149627

# IMD3005_3    >>>         PARTY IDENTIFICATION: WHO
# 0080001. ALB - Socialist Party                               240
# 0080002. ALB - Democratic Party                              388
#                                                            ...  
# 9999988. NONE OF THE CANDIDATES PARTIES                      517
# 9999989. INDEPENDENT CANDIDATE                                34
# 9999990. OTHER LEFT WING CANDIDATE/PARTY                      67
# 9999991. OTHER RIGHT WING CANDIDATE/PARTY                     36
# 9999992. OTHER CANDIDATE/PARTY (NOT FURTHER SPECIFIED)      1380
# 9999997. VOLUNTEERED: REFUSED                               1840
# 9999998. VOLUNTEERED: DON'T KNOW                            1304
# 9999999. MISSING                                          120168

# IMD3005_4    >>>         PARTY IDENTIFICATION: HOW CLOSE
# 1. VERY CLOSE                  31156
# 2. SOMEWHAT CLOSE              77102
# 3. NOT VERY CLOSE              33777
# 7. VOLUNTEERED: REFUSED          756
# 8. VOLUNTEERED: DON'T KNOW      1868
# 9. MISSING                    136424



In [301]:
# the order in which/wording of these questions are asked varies between studies/modules a little
# also, not everyone knows what left-right means (altho sometimes respondents said they didn't know
# in one of the variable but still used it in the others!)
# some of the base modules also have an "alternative scale" (for when people in a region didn't understand 'left-right')
# but that's not used for IMD

# IMD3006      >>>         LEFT-RIGHT - SELF 
# 00. LEFT                                         9205
# 1                                                6106
# 2                                               11270
# 3                                               17574
# 4                                               19048
# 5                                               61715
# 6                                               21006
# 7                                               23262
# 8                                               20805
# 9                                                7978
# 10. RIGHT                                       17835
# 95. VOLUNTEERED: HAVEN'T HEARD OF LEFT-RIGHT     5311
# 97. VOLUNTEERED: REFUSED                         4167
# 98. VOLUNTEERED: DON'T KNOW WHERE TO PLACE      31655
# 99. MISSING                                     24146

# IMD3007_A    >>>         LEFT-RIGHT - PARTY A
# 00. LEFT                                        15509
# 1                                                7935
# 2                                               12120
# 3                                               16338
# 4                                               17560
# 5                                               31050
# 6                                               17454
# 7                                               20695
# 8                                               24041
# 9                                               15368
# 10. RIGHT                                       30262
# 95. VOLUNTEERED: HAVEN'T HEARD OF LEFT-RIGHT     5062
# 96. VOLUNTEERED: HAVEN'T HEARD OF PARTY          2839
# 97. VOLUNTEERED: REFUSED                         2900
# 98. VOLUNTEERED: DON'T KNOW WHERE TO PLACE      29791
# 99. MISSING                                     32159
# Same but includes "VOLUNTEERED: HAVEN'T HEARD OF PARTY"

## and so on, but with higher MISSING for later (less known) parties and GHI (not in all modules)

# IMD3007_B    >>>         LEFT-RIGHT - PARTY B
# IMD3007_C    >>>         LEFT-RIGHT - PARTY C
# IMD3007_D    >>>         LEFT-RIGHT - PARTY D
# IMD3007_E    >>>         LEFT-RIGHT - PARTY E
# IMD3007_F    >>>         LEFT-RIGHT - PARTY F
# IMD3007_G    >>>         LEFT-RIGHT - PARTY G (OPTIONAL)
# IMD3007_H    >>>         LEFT-RIGHT - PARTY H (OPTIONAL)
# IMD3007_I    >>>         LEFT-RIGHT - PARTY I (OPTIONAL)



In [307]:
# same point as above, but they mention cases where respondents score all the parties the same/don't know to all of them

# IMD3008_A    >>>         LIKE-DISLIKE - PARTY A
# 00. STRONGLY DISLIKE                                    27417
# 1                                                       11605
# 2                                                       14335
# 3                                                       17187
# 4                                                       17711
# 5                                                       40282
# 6                                                       21798
# 7                                                       26454
# 8                                                       30162
# 9                                                       17083
# 10. STRONGLY LIKE                                       28921
# 96. HAVEN'T HEARD OF PARTY                               2246
# 97. VOLUNTEERED: REFUSED                                 2631
# 98. DON'T KNOW ENOUGH ABOUT/DON'T KNOW WHERE TO RATE    10148
# 99. MISSING                                             13103

# IMD3008_B    >>>         LIKE-DISLIKE - PARTY B
# IMD3008_C    >>>         LIKE-DISLIKE - PARTY C
# IMD3008_D    >>>         LIKE-DISLIKE - PARTY D
# IMD3008_E    >>>         LIKE-DISLIKE - PARTY E
# IMD3008_F    >>>         LIKE-DISLIKE - PARTY F
# IMD3008_G    >>>         LIKE-DISLIKE - PARTY G (OPTIONAL)
# IMD3008_H    >>>         LIKE-DISLIKE - PARTY H (OPTIONAL)
# IMD3008_I    >>>         LIKE-DISLIKE - PARTY I (OPTIONAL)
# 00. STRONGLY DISLIKE                                      6160
# 1                                                         3607
# 2                                                         3865
# 3                                                         3601
# 4                                                         3244
# 5                                                         5677
# 6                                                         2252
# 7                                                         1511
# 8                                                         1031
# 9                                                          451
# 10. STRONGLY LIKE                                          594
# 96. HAVEN'T HEARD OF PARTY                                3738
# 97. VOLUNTEERED: REFUSED                                   847
# 98. DON'T KNOW ENOUGH ABOUT/DON'T KNOW WHERE TO RATE      4387
# 99. MISSING                                             240118


# IMD3009_A    >>>         LIKE-DISLIKE - LEADER A
# 00. STRONGLY DISLIKE                                    23001
# 1                                                        9477
# 2                                                       10739
# 3                                                       11762
# 4                                                       12356
# 5                                                       26758
# 6                                                       16015
# 7                                                       20595
# 8                                                       24614
# 9                                                       15445
# 10. STRONGLY LIKE                                       24192
# 96. HAVEN'T HEARD OF LEADER                              2338
# 97. VOLUNTEERED: REFUSED                                 1787
# 98. DON'T KNOW ENOUGH ABOUT/DON'T KNOW WHERE TO RATE     6729
# 99. MISSING                                             75275


#Respondents outside of a region where a party contested the
#  | election were not asked to evaluate its leaders. These
#  | respondents are coded as 97 in CSES Module 1 but were re-coded  
#  | to “99. MISSING” for IMD. This applies to the Scottish National
#  | Party, SNP (PARTY D) and Plaid Cymru, PC (PARTY E). 

# IMD3009_B    >>>         LIKE-DISLIKE - LEADER B
# IMD3009_C    >>>         LIKE-DISLIKE - LEADER C
# IMD3009_D    >>>         LIKE-DISLIKE - LEADER D
# IMD3009_E    >>>         LIKE-DISLIKE - LEADER E
# IMD3009_F    >>>         LIKE-DISLIKE - LEADER F
# IMD3009_G    >>>         LIKE-DISLIKE - LEADER G (OPTIONAL)
# IMD3009_H    >>>         LIKE-DISLIKE - LEADER H (OPTIONAL)
# IMD3009_I    >>>         LIKE-DISLIKE - LEADER I (OPTIONAL)
# 00. STRONGLY DISLIKE                                      5832
# 1                                                         3299
# 2                                                         3344
# 3                                                         3470
# 4                                                         3047
# 5                                                         5616
# 6                                                         2559
# 7                                                         2387
# 8                                                         1576
# 9                                                          823
# 10. STRONGLY LIKE                                          906
# 96. HAVEN'T HEARD OF LEADER                               5723
# 97. VOLUNTEERED: REFUSED                                   498
# 98. DON'T KNOW ENOUGH ABOUT/DON'T KNOW WHERE TO RATE      2943
# 99. MISSING                                             239060



In [339]:
# IMD3010      >>>         SATISFACTION WITH DEMOCRACY
# 1. VERY SATISFIED                         26656
# 2. FAIRLY SATISFIED                      126450
# 4. NOT VERY SATISFIED                     77236
# 5. NOT AT ALL SATISFIED                   28960
# 6. NEITHER SATISFIED NOR DISSATISFIED      3060
# 7. VOLUNTEERED: REFUSED                    1200
# 8. VOLUNTEERED: DON'T KNOW                 9874
# 9. MISSING                                 7647

# IMD3011      >>>         EFFICACY: WHO IS IN POWER CAN MAKE A DIFFERENCE
# 1. IT DOESN'T MAKE ANY DIFFERENCE WHO IS IN POWER     23006
# 2                                                     19891
# 3                                                     44526
# 4                                                     62264
# 5. IT MAKES A BIG DIFFERENCE WHO IS IN POWER         103766
# 7. VOLUNTEERED: REFUSED                                1014
# 8. VOLUNTEERED: DON'T KNOW                             5791
# 9. MISSING                                            20825

# IMD3012      >>>         EFFICACY: WHO PEOPLE VOTE FOR MAKES A DIFFERENCE
# 1. WHO PEOPLE VOTE FOR WON'T MAKE ANY DIFFERENCE     20689
# 2                                                    18922
# 3                                                    42765
# 4                                                    65063
# 5. WHO PEOPLE VOTE FOR CAN MAKE A BIG DIFFERENCE    105384
# 7. VOLUNTEERED: REFUSED                               1055
# 8. VOLUNTEERED: DON'T KNOW                            5772
# 9. MISSING                                           21433

# Weird - IMD3013_1 was supposed to be a filter ques for IMD3013_2 and IMD3013_3
# (only ppl who answer BETTER/WORSE get asked those, respectively)
# But some studies asked them anyway if they'd said the opposite - and some didn't ask those questions

# IMD3013_1    >>>         STATE OF ECONOMY (OVER PAST 12 MONTHS)
# 1. GOTTEN BETTER               28461
# 3. STAYED THE SAME             54939
# 5. GOTTEN WORSE                45141
# 7. VOLUNTEERED: REFUSED          220
# 8. VOLUNTEERED: DON'T KNOW      3760
# 9. MISSING                    148562

# IMD3013_2    >>>         STATE OF ECONOMY - BETTER
# 1. MUCH BETTER                  3775
# 2. SOMEWHAT BETTER             22676
# 7. VOLUNTEERED: REFUSED           27
# 8. VOLUNTEERED: DON'T KNOW       331
# 9. MISSING                    254274

# IMD3013_3    >>>         STATE OF ECONOMY - WORSE
# 4. SOMEWHAT WORSE              23383
# 5. MUCH WORSE                  17055
# 7. VOLUNTEERED: REFUSED          191
# 8. VOLUNTEERED: DON'T KNOW       506
# 9. MISSING                    239948

# Now thinking about the performance of the [government in 
#  [CAPITAL]/president] in general, how good or bad a job do you 
#  think the [government/president in [CAPITAL]] has done over the 
#  past [NUMBER OF YEARS SINCE LAST GOVERNMENT TOOK OFFICE, BEFORE 
#  THE CURRENT ELECTION] years? Has [it/he/she] done a very good job?
#  A good job? A bad job? A very bad job?

# IMD3014      >>>         GOVERNMENT PERFORMANCE: GENERAL
# 1. VERY GOOD JOB                6975
# 2. GOOD JOB                    60204
# 3. BAD JOB                     42119
# 4. VERY BAD JOB                15726
# 6. NEITHER GOOD NOR BAD         2943
# 7. VOLUNTEERED: REFUSED         1043
# 8. VOLUNTEERED: DON'T KNOW      6445
# 9. MISSING                    145628

# These are manually created per study, so they vary in difficulty
# Only recognise explicit correct/incorrect -> refuse/DK handled separately
# Very large bunch of studies didn't have a DK option!
# PAGES of notes here!

# IMD3015_1    >>>         DICHOTOMIZED POLITICAL INFORMATION ITEM - 1ST   
# 0. INCORRECT                   34061
# 1. CORRECT                    161507
# 7. VOLUNTEERED: REFUSED         1431
# 8. VOLUNTEERED: DON'T KNOW     40100
# 9. MISSING                     43984

# IMD3015_2    >>>         DICHOTOMIZED POLITICAL INFORMATION ITEM - 2ND   
# 0. INCORRECT                   67891
# 1. CORRECT                    113853
# 7. VOLUNTEERED: REFUSED         2070
# 8. VOLUNTEERED: DON'T KNOW     51209
# 9. MISSING                     46060

# IMD3015_3    >>>         DICHOTOMIZED POLITICAL INFORMATION ITEM - 3RD 
# 0. INCORRECT                  73679
# 1. CORRECT                    97525
# 7. VOLUNTEERED: REFUSED        1442
# 8. VOLUNTEERED: DON'T KNOW    56567
# 9. MISSING                    51870

# IMD3015_4    >>>         DICHOTOMIZED POLITICAL INFORMATION ITEM - 4TH     
# 0. INCORRECT                   11781
# 1. CORRECT                     23765
# 7. VOLUNTEERED: REFUSED         1106
# 8. VOLUNTEERED: DON'T KNOW     30155
# 9. MISSING                    214276


# Built using the above items in each 

# IMD3015_A    >>>         POLITICAL KNOWLEDGE SCALE - CSES MODULE 1 (0-3 SCALE)
# 0               4963
# 1              10273
# 2              11441
# 3               9120
# 9. MISSING    245286

# IMD3015_B    >>>         POLITICAL KNOWLEDGE SCALE - CSES MODULE 2 (0-3 SCALE)     
# 0               7913
# 1              14337
# 2              17495
# 3              13535
# 9. MISSING    227803

# IMD3015_C    >>>         POLITICAL KNOWLEDGE SCALE - CSES MODULE 3 (0-3 SCALE)
# 0              12522
# 1              18473
# 2              23453
# 3              14622
# 9. MISSING    212013

# IMD3015_D    >>>         POLITICAL KNOWLEDGE SCALE - CSES MODULE 4 (0-4 SCALE)
# 0              11124
# 1              13646
# 2              14363
# 3              13501
# 4               7364
# 9. MISSING    221085

# CSES_int["IMD3013_3"].cat.remove_unused_categories().value_counts().sort_index()

4. SOMEWHAT WORSE              23383
5. MUCH WORSE                  17055
7. VOLUNTEERED: REFUSED          191
8. VOLUNTEERED: DON'T KNOW       506
9. MISSING                    239948
Name: IMD3013_3, dtype: int64

In [350]:
# IMD5000_A    >>>         PARTY A IDENTIFIER - NUMERICAL
# 7560001. CHE - Swiss People's Party                             11021
# 5780001. NOR - Labour Party                                      9628
# 4840001. MEX - Institutional Revolutionary Party                 8030
# 1240002. CAN - Conservative Party                                7953
# 5540001. NZL - National Party                                    7827
#                                                                 ...  
# 2330001. EST - Estonian Reform Party                             1000
# 2500008. FRA - Rally For The Republic                            1000
# 1000001. BGR - Citizens for European Development of Bulgaria      999
# 8580001. URY - Broad Front                                        968
# 4990011. MNE - Coalition 'For a European Montenegro'              967

# IMD5000_B    >>>         PARTY B IDENTIFIER - NUMERICAL
# IMD5000_C    >>>         PARTY C IDENTIFIER - NUMERICAL
# 9999999. MISSING entries start here and increase as you work down

# IMD5000_D    >>>         PARTY D IDENTIFIER - NUMERICAL
# IMD5000_E    >>>         PARTY E IDENTIFIER - NUMERICAL
# IMD5000_F    >>>         PARTY F IDENTIFIER - NUMERICAL
# IMD5000_G    >>>         PARTY G IDENTIFIER - NUMERICAL
# IMD5000_H    >>>         PARTY H IDENTIFIER - NUMERICAL
# IMD5000_I    >>>         PARTY I IDENTIFIER - NUMERICAL
# 9999999. MISSING                                        210169
# 5280007. NLD - Reformed Political Party                   6086
# 0760009. BRA - Democrats                                  3136
# 5540006. NZL - United Future New Zealand                  2598
# 0760046. BRA - Liberal Party                              2514
# 7520009. SWE - Feminist Party                             2379
# 6420020. ROU - Democratic Liberal Party                   2283
# 0560013. BEL - Confederated Ecologists                    2225
# 2080011. DNK - Centre Democrats                           2026
# 2500018. FRA - Radical Party of the Left                  2000
# 0760010. BRA - Brazilian Labor Party                      2000
# 4840026. MEX - Mexican Liberal Party                      1991
# 3920005. JPN - Green Wind                                 1937
# 8400006. USA - Constitution Party                         1929
# 2760009. DEU - National Democratic Party Of Germany       1889
# 6160007. POL - Labour Union                               1794
# 5780009. NOR - Red Party                                  1727
# 2030012. CZE - Czech Pirate Party                         1653
# 6040009. PER - Hope Front                                 1572
# 7640009. THA - Power of Choburi Party                     1500
# 6430042. RUS - Rebirth of Russia Party                    1496
# 2080008. DNK - Christian Democrats                        1442
# 0320009. ARG - Justicialist Party                         1406
# 6200026. PRT - Humanist Party                             1303
# 7100009. ZAF - African Christian Democratic Party         1300
# 7240012. ESP - Union, Progress and Democracy              1204
# 6080009. PHL - People's Reform Party                      1200
# 6080087. PHL - Alliance for the Common Good               1200
# 4040005. KEN - Kenya African National Union               1200
# 1520007. CHL - Radical Social-democratic Party            1200
# 4840004. MEX - Labor Party                                1197
# 7030004. SVK - Christian Democratic Movement              1150
# 0080010. ALB - Christian Democratic Party                 1116
# 6420301. ROU - Monica-Luisa Macovei (Independent)         1112
# 7920012. TUR - Democratic Left Party                      1109
# 3440009. HKG - Liberal Party                              1044
# 3760010. ISR - United Arab List                           1017
# 4400007. LTU - Nationalist Party - 'Young Lithuania'      1009
# 1910011. HRV - Istrian Democratic Assembly                1004
# 2500004. FRA - French Communist Party                     1000
# 1120017. BLR - Christian Conservative Party               1000
# 0400011. AUT - Pirate Party                               1000
# 1000009. BGR - Movement 21                                 999
# 4990007. MNE - Croatian Civic Initiative                   967

# CSES_int["IMD5000_A"].cat.remove_unused_categories().value_counts()
# (CSES_int["IMD5000_E"]=="9999999. MISSING").sum()

In [ ]:
# IMD5001_A    >>>         PERCENT VOTE - LOWER HOUSE - PARTY A
# 4.27                                             1000
# 13.94                                            3136
# 14.6                                             1000
# 15.36                                            2225
# 16.9                                             2000
#                                                 ...  
# 58.63                                            2283
# 62.15                                            1300
# 65.9                                             1200
# 997. NOT APPLICABLE: NO LOWER HOUSE ELECTION    22970
# 999. MISSING                                     1200

# IMD5001_B    >>>         PERCENT VOTE - LOWER HOUSE - PARTY B
# IMD5001_C    >>>         PERCENT VOTE - LOWER HOUSE - PARTY C
# IMD5001_D    >>>         PERCENT VOTE - LOWER HOUSE - PARTY D
# IMD5001_E    >>>         PERCENT VOTE - LOWER HOUSE - PARTY E
# IMD5001_F    >>>         PERCENT VOTE - LOWER HOUSE - PARTY F
# IMD5001_G    >>>         PERCENT VOTE - LOWER HOUSE - PARTY G
# IMD5001_H    >>>         PERCENT VOTE - LOWER HOUSE - PARTY H
# IMD5001_I    >>>         PERCENT VOTE - LOWER HOUSE - PARTY I

In [ ]:
# IMD5002_A    >>>         PERCENT SEATS - LOWER HOUSE - PARTY A
# 6.36                                             1000
# 13.45                                            3136
# 16.67                                            2225
# 17.2                                             2000
# 17.3                                             1000
#                                                 ...  
# 62.25                                            1300
# 65.21                                            2897
# 66.0                                             1200
# 66.26                                            2283
# 997. NOT APPLICABLE: NO LOWER HOUSE ELECTION    24170

# IMD5002_B    >>>         PERCENT SEATS - LOWER HOUSE - PARTY B
# IMD5002_C    >>>         PERCENT SEATS - LOWER HOUSE - PARTY C
# IMD5002_D    >>>         PERCENT SEATS - LOWER HOUSE - PARTY D
# IMD5002_E    >>>         PERCENT SEATS - LOWER HOUSE - PARTY E
# IMD5002_F    >>>         PERCENT SEATS - LOWER HOUSE - PARTY F
# IMD5002_G    >>>         PERCENT SEATS - LOWER HOUSE - PARTY G
# IMD5002_H    >>>         PERCENT SEATS - LOWER HOUSE - PARTY H
# IMD5002_I    >>>         PERCENT SEATS - LOWER HOUSE - PARTY I

In [ ]:
# IMD5003_A    >>>         PERCENT VOTE - UPPER HOUSE - PARTY A
# 15.38                                             2225
# 16.96                                             3136
# 19.2                                              1000
#                                 ....
# 51.66                                             2102
# 53.41                                             1929
# 60.58                                             2283
# 996. NOT APPLICABLE: UNICAMERAL SYSTEM          104155
# 997. NOT APPLICABLE: NO UPPER HOUSE ELECTION    102462
# 999. MISSING                                     14356

# IMD5003_B    >>>         PERCENT VOTE - UPPER HOUSE - PARTY B
# IMD5003_C    >>>         PERCENT VOTE - UPPER HOUSE - PARTY C
# IMD5003_D    >>>         PERCENT VOTE - UPPER HOUSE - PARTY D
# IMD5003_E    >>>         PERCENT VOTE - UPPER HOUSE - PARTY E
# IMD5003_F    >>>         PERCENT VOTE - UPPER HOUSE - PARTY F
# IMD5003_G    >>>         PERCENT VOTE - UPPER HOUSE - PARTY G
# IMD5003_H    >>>         PERCENT VOTE - UPPER HOUSE - PARTY H
# IMD5003_I    >>>         PERCENT VOTE - UPPER HOUSE - PARTY I

In [ ]:
# IMD5004_A    >>>         PERCENT SEATS - UPPER HOUSE - PARTY A
# 7.41                                              3136
# 14.8                                              1000
# 15.2                                              5212
# 17.39                                             1418
#                                 ...
# 69.32                                             2283
# 69.7                                              1929
# 75.0                                              1794
# 996. NOT APPLICABLE: UNICAMERAL SYSTEM          104155
# 997. NOT APPLICABLE: NO UPPER HOUSE ELECTION     98632

# IMD5004_B    >>>         PERCENT SEATS - UPPER HOUSE - PARTY B
# IMD5004_C    >>>         PERCENT SEATS - UPPER HOUSE - PARTY C
# IMD5004_D    >>>         PERCENT SEATS - UPPER HOUSE - PARTY D
# IMD5004_E    >>>         PERCENT SEATS - UPPER HOUSE - PARTY E
# IMD5004_F    >>>         PERCENT SEATS - UPPER HOUSE - PARTY F
# IMD5004_G    >>>         PERCENT SEATS - UPPER HOUSE - PARTY G
# IMD5004_H    >>>         PERCENT SEATS - UPPER HOUSE - PARTY H
# IMD5004_I    >>>         PERCENT SEATS - UPPER HOUSE - PARTY I

In [358]:
# IMD5005_A    >>>         PERCENT VOTE - PRESIDENT - PARTY A
# 0.0                                                1000
# 19.88                                              1000
# 23.23                                              1200
# 23.45                                              1200
#                             ...
# 58.45                                              1905
# 71.91                                              1496
# 88.72                                              2000
# 996. NOT APPLICABLE: NO ROLE OF PRESIDENT        103216
# 997. NOT APPLICABLE: NO PRESIDENTIAL ELECTION    119063
# 999. MISSING                                       2200

# IMD5005_B    >>>         PERCENT VOTE - PRESIDENT - PARTY B
# IMD5005_C    >>>         PERCENT VOTE - PRESIDENT - PARTY C
# IMD5005_D    >>>         PERCENT VOTE - PRESIDENT - PARTY D
# IMD5005_E    >>>         PERCENT VOTE - PRESIDENT - PARTY E
# IMD5005_F    >>>         PERCENT VOTE - PRESIDENT - PARTY F
# IMD5005_G    >>>         PERCENT VOTE - PRESIDENT - PARTY G
# IMD5005_H    >>>         PERCENT VOTE - PRESIDENT - PARTY H
# IMD5005_I    >>>         PERCENT VOTE - PRESIDENT - PARTY I


In [ ]:
# IMD5006_1    >>>         ELECTORAL TURNOUT - TURNOUT AS A PERCENTAGE OF REGISTERED VOTERS (ER)
# 40.57           2402
# 41.68           1991
# 41.76           2283
# 43.3            2048
# 43.57            674
#                 ... 
# 93.23           3953
# 94.32           1769
# 94.76           1873
# 95.77           1798
# 999. MISSING    3759

# IMD5006_2    >>>         ELECTORAL TURNOUT - TURNOUT AS A PERCENTAGE OF THE VOTING AGE POPULATION (VAP)
# 26.24            815
# 30.19           1044
# 34.94           2048
# 37.27           1418
# 38.3            4391
#                 ... 
# 87.4            1570
# 88.93           1572
# 89.12           1446
# 96.87            968
# 999. MISSING    7618

# IMD5007      >>>         COMPULSORY VOTING
# 1. YES; STRICTLY ENFORCED SANCTIONS        22087
# 2. YES; WEAKLY ENFORCED SANCTIONS          21585
# 3. YES; WITHOUT SANCTION FOR VIOLATION     19677
# 5. NO                                     217734

In [366]:
# IMD5008_1    >>>         PARTY OF THE PRIME MINISTER BEFORE
# 0080001. ALB - Socialist Party                        1116
# 0360001. AUS - Liberal Party                          3642
# 0360003. AUS - Australian Labor Party                 5751
# 0400001. AUT - Social Democratic Party of Austria     2165
# 0560003. BEL - Christian Democratic & Flemish         4139
#                                                      ...  
# 8040015. UKR - People's-Democratic Party              1148
# 8260001. GBR - Conservatives                          4464
# 8260002. GBR - Labour                                  860
# 9999989. INDEPENDENT CANDIDATE                       13430
# 9999997. NOT APPLICABLE                              55842

# IMD5008_2    >>>         PARTY OF THE PRIME MINISTER AFTER
# 0080002. ALB - Democratic Party                       1116
# 0360001. AUS - Liberal Party                          7520
# 0360003. AUS - Australian Labor Party                 1873
# 0400001. AUT - Social Democratic Party of Austria     2165
# 0560004. BEL - Flemish Liberals and Democrats         6364
#                                                      ...  
# 8040015. UKR - People's-Democratic Party              1148
# 8260001. GBR - Conservatives                          1567
# 8260002. GBR - Labour                                 3757
# 9999989. INDEPENDENT CANDIDATE                        9664
# 9999997. NOT APPLICABLE                              57042

# IMD5009_1    >>>         PARTY OF THE PRESIDENT BEFORE
# 0320001. ARG - Front for the Victory                              1406
# 0400001. AUT - Social Democratic Party of Austria                 2165
# 0760001. BRA - Workers Party                                      6136
# 0760002. BRA - Brazilian Social Dmeocratic Party                  2514
# 1000001. BGR - Citizens for European Development of Bulgaria       999
#                                                                  ...  
# 8400001. USA - Republican Party                                   3168
# 8400002. USA - Democratic Party                                   3463
# 8580001. URY - Broad Front                                         968
# 9999989. INDEPENDENT CANDIDATE                                   24619
# 9999997. NOT APPLICABLE                                         105366

# IMD5009_2    >>>         PARTY OF THE PRESIDENT AFTER
# 0320002. ARG - Let's Change                                       1406
# 0400001. AUT - Social Democratic Party of Austria                 2165
# 0760001. BRA - Workers Party                                      8650
# 1000001. BGR - Citizens for European Development of Bulgaria       999
# 1000034. BGR - United Democratic Forces                           1482
#                                                                  ...  
# 8400001. USA - Republican Party                                   1066
# 8400002. USA - Democratic Party                                   5565
# 8580001. URY - Broad Front                                         968
# 9999989. INDEPENDENT CANDIDATE                                   25605
# 9999997. NOT APPLICABLE                                         105366


In [371]:
# both these variables are based on expert surveys, they shift with shifts in opinions/shifts of the party
# module 2 has a more detail categorisation
# "religious parties" only a category in mod 1 and 2

# IMD5011_A    >>>         IDEOLOGICAL FAMILY - PARTY A
# 2. COMMUNIST PARTIES                    4990
# 3. SOCIALIST PARTIES                   13312
# 4. SOCIAL DEMOCRATIC PARTIES           77226
# 5. LEFT LIBERAL PARTIES                15138
# 6. LIBERAL PARTIES                     28017
# 7. RIGHT LIBERAL PARTIES               26709
# 8. CHRISTIAN DEMOCRATIC PARTIES        17742
# 9. CONSERVATIVE PARTIES                55517
# 10. NATIONAL PARTIES                   17193
# 11. AGRARIAN PARTIES                    2870
# 12. ETHNIC PARTIES                      2040
# 14. INDEPENDENT PARTIES                 2220
# 18. SOCIAL-LIBERAL                      1482
# 22. NATIONALIST LEFT                    2032
# 90. OTHER                               5347
# 97. NOT APPLICABLE                      2757
# 98. NO IDEOLOGICAL FAMILY MENTIONED     1500
# 99. MISSING                             4991

# IMD5011_B    >>>         IDEOLOGICAL FAMILY - PARTY B
# IMD5011_C    >>>         IDEOLOGICAL FAMILY - PARTY C
# IMD5011_D    >>>         IDEOLOGICAL FAMILY - PARTY D
# IMD5011_E    >>>         IDEOLOGICAL FAMILY - PARTY E
# IMD5011_F    >>>         IDEOLOGICAL FAMILY - PARTY F
# IMD5011_G    >>>         IDEOLOGICAL FAMILY - PARTY G
# IMD5011_H    >>>         IDEOLOGICAL FAMILY - PARTY H
# IMD5011_I    >>>         IDEOLOGICAL FAMILY - PARTY I 
# 1. ECOLOGY PARTIES                       5362
# 2. COMMUNIST PARTIES                     1000
# 3. SOCIALIST PARTIES                     4718
# 4. SOCIAL DEMOCRATIC PARTIES             2108
# 5. LEFT LIBERAL PARTIES                  1991
# 6. LIBERAL PARTIES                       5002
# 7. RIGHT LIBERAL PARTIES                 4443
# 8. CHRISTIAN DEMOCRATIC PARTIES          7291
# 9. CONSERVATIVE PARTIES                  9739
# 10. NATIONAL PARTIES                      967
# 12. ETHNIC PARTIES                       1017
# 13. REGIONAL PARTIES                     1004
# 14. INDEPENDENT PARTIES                  1572
# 26. ORTHODOX-CALVINIST                   1574
# 90. OTHER                                8740
# 97. NOT APPLICABLE                       1112
# 98. NO IDEOLOGICAL FAMILY MENTIONED      1500
# 99. MISSING                            221943

# as above, but the original module also has an 'alternative scale' for studies where people don't think left-right works

# also WTF! Why not fix this???
#  | [ELECTION STUDY NOTES] - UNITED STATES (2012): IMD5012_
#  |
#  | The alphabetical codes assigned to parties G, H, and I for the 
#  | UNITED STATES 2012 do not match the letter in this variable. 
#  | Data for PARTY G (Libertarian Party, LP) is coded in IMD5012_C,
#  | data for PARTY H (Green Party, GPUS) is coded in IMD5012_D, and
#  | data for PARTY I (Constitution Party, Con) is coded in 
#  | IMD5012_E.

# IMD5012_A    >>>         LEFT-RIGHT - PARTY A
# 00. LEFT               2000
# 1.0                    4738
# 2.0                    4141
# 3.0                   44590
# 4.0                   44562
# 4.5                    2971
# 5.0                   36751
# 6.0                   37612
# 7.0                   62527
# 8.0                   16402
# 9.0                   11332
# 97. NOT APPLICABLE     1009
# 99. MISSING           12448

# IMD5012_B    >>>         LEFT-RIGHT - PARTY B
# IMD5012_C    >>>         LEFT-RIGHT - PARTY C
# IMD5012_D    >>>         LEFT-RIGHT - PARTY D
# IMD5012_E    >>>         LEFT-RIGHT - PARTY E
# IMD5012_F    >>>         LEFT-RIGHT - PARTY F
# IMD5012_G    >>>         LEFT-RIGHT - PARTY G
# IMD5012_H    >>>         LEFT-RIGHT - PARTY H
# IMD5012_I    >>>         LEFT-RIGHT - PARTY I 
# 1                6735
# 2                2135
# 3                4162
# 4                9674
# 5                5464
# 6               10182
# 7                3197
# 8               10181
# 9                2774
# 10. RIGHT        1929
# 99. MISSING    224650

In [ ]:
# IMD5013      >>>         ELECTORAL FORMULA IN ALL SEGMENTS: LOWER HOUSE
# 1. MAJORITARIAN     48842
# 2. PROPORTIONAL    157808
# 3. MIXED            74433

# IMD5014      >>>         ELECTORAL FORMULA: PRESIDENTIAL ELECTION
# 1. PLURALITY                   32385
# 2. ABSOLUTE MAJORITY RULE      84511
# 3. QUALIFIED MAJORITY RULE      1406
# 4. ELECTORAL COLLEGE            6631
# 5. ALTERNATIVE VOTE             5655
# 7. NOT APPLICABLE             147515
# 9. MISSING                      2980


# IMD5016_1    >>>         VOTES CAST - LOWER 1ST SEGMENT (TIER) (How many votes do voters cast or can cast?)
#      In systems where voters rank order the candidates, if there
#      are 10 candidates (for example), the response to this
#      question should be 10.
# 1                                 257577
# 2                                   1102
# 91. [SEE ELECTION STUDY NOTES]     22404

# IMD5016_2    >>>         VOTES CAST - LOWER 2ND SEGMENT (TIER)
# 1                                  60498
# 91. [SEE ELECTION STUDY NOTES]      9628
# 97. NOT APPLICABLE                210957

# IMD5016_3    >>>         VOTES CAST - UPPER 1ST SEGMENT (TIER)
# 1                                  58166
# 2                                   4514
# 3                                   4836
# 12                                  3600
# 91. [SEE ELECTION STUDY NOTES]     30349
# 97. NOT APPLICABLE                179618

# IMD5016_4    >>>         VOTES CAST - UPPER 2ND SEGMENT (TIER)
# 1                       8053
# 97. NOT APPLICABLE    273030

# IMD5017_1    >>>         VOTING PROCEDURE - LOWER - 1ST SEGMENT (TIER) (candidates or party lists)
#      Do they vote for candidates (not party lists) or party
#      lists?
#      Definition: Party bloc voting is used in multi-member
#      districts where voters cast a single party-centered vote for
#      their party of choice; the party with the most votes wins
#      all of the district seats.

# 1. CANDIDATES                    131930
# 2. PARTY LISTS                   115011
# 4. [SEE ELECTION STUDY NOTES]     34142

# IMD5017_2    >>>         VOTING PROCEDURE - LOWER - 2ND SEGMENT (TIER)
# 1. CANDIDATES          1200
# 2. PARTY LISTS        82304
# 7. NOT APPLICABLE    197579

# IMD5017_3    >>>         VOTING PROCEDURE - UPPER - 1ST SEGMENT (TIER)
# 1. CANDIDATES          1200
# 2. PARTY LISTS        82304
# 7. NOT APPLICABLE    197579

# IMD5017_4    >>>         VOTING PROCEDURE - UPPER - 2ND SEGMENT (TIER)
# 2. PARTY LISTS        22631
# 7. NOT APPLICABLE    258452

# IMD5018_1    >>>         VOTING ROUNDS - LOWER - 1ST SEGMENT (TIER) (How many rounds of voting are there?)
# 1                     269773
# 2                       6023
# 97. NOT APPLICABLE      5287

# IMD5018_2    >>>         VOTING ROUNDS - LOWER - 2ND SEGMENT (TIER)
# 1                      85325
# 97. NOT APPLICABLE    195758

# IMD5018_3    >>>         VOTING ROUNDS - UPPER - 1ST SEGMENT (TIER)
# 1                      89117
# 2                      18710
# 97. NOT APPLICABLE    173256

# IMD5018_4    >>>         VOTING ROUNDS - UPPER - 2ND SEGMENT (TIER)
# 1                      26468
# 97. NOT APPLICABLE    254615

# IMD5021_1    >>>         IS THERE PARTY THRESHOLD - LOWER - 1ST SEGMENT (TIER)
#  Are there legally mandated thresholds that a party must
#  exceed before it is eligible to receive seats?
# 1. YES         61195
# 5. NO          94526
# 9. MISSING    125362

# IMD5021_2    >>>         IS THERE PARTY THRESHOLD - LOWER - 2ND SEGMENT (TIER)
# 1. YES                39269
# 5. NO                  9044
# 7. NOT APPLICABLE    107408
# 9. MISSING           125362

# IMD5021_3    >>>         IS THERE PARTY THRESHOLD - UPPER - 1ST SEGMENT (TIER)
# 1. YES                 9312
# 5. NO                 47938
# 7. NOT APPLICABLE     98471
# 9. MISSING           125362

# IMD5021_4    >>>         IS THERE PARTY THRESHOLD - UPPER - 2ND SEGMENT (TIER)
# 1. YES                 7588
# 5. NO                  3310
# 7. NOT APPLICABLE    144823
# 9. MISSING           125362

# IMD5022_1    >>>         PARTY THRESHOLD - LOWER - 1ST SEGMENT (TIER)
# If YES in Question IMD5021, what is the threshold?
# (Quite a bit of fiddly exceptions/complications here)
# 00.00 THERE IS NO THRESHOLD                          94526
# 0.7                                                   4512
# 2.0                                                   2217
# 3.0                                                   6636
# 4.0                                                   7629
# 5.0                                                  20620
# 10.0                                                  2195
# 16.67                                                 1937
# 17.0                                                  1373
# 96.00 OTHER THRESHOLD [SEE ELECTION STUDY NOTES]     14076
# 99.00 MISSING                                       125362

# IMD5022_2    >>>         PARTY THRESHOLD - LOWER - 2ND SEGMENT (TIER)
# 00.00 THERE IS NO THRESHOLD                           9044
# 2                                                    11430
# 4                                                     7900
# 5                                                     8190
# 96.00 OTHER THRESHOLD [SEE ELECTION STUDY NOTES]     11749
# 97.00 NOT APPLICABLE                                107408
# 99.00 MISSING                                       125362

# IMD5022_3    >>>         PARTY THRESHOLD - UPPER - 1ST SEGMENT (TIER)
# 00.00 THERE IS NO THRESHOLD                          45332
# 3                                                     1204
# 96.00 OTHER THRESHOLD [SEE ELECTION STUDY NOTES]      8108
# 97.00 NOT APPLICABLE                                101077
# 99.00 MISSING                                       125362

# IMD5022_4    >>>         PARTY THRESHOLD - UPPER - 2ND SEGMENT (TIER)
# 00.00 THERE IS NO THRESHOLD      3310
# 2                                7588
# 97.00 NOT APPLICABLE           144823
# 99.00 MISSING                  125362

# On what date was the election originally scheduled to be
# held? (not present mod1&2)
# On what date was the election actually held?
# IMD5024_1    >>>         DATE ELECTION SCHEDULED - MONTH

# 01. JANUARY        1826
# 02. FEBRUARY       1442
# 03. MARCH          7509
# 04. APRIL         12720
# 05. MAY           18614
# 06. JUNE          11258
# 07. JULY          12230
# 08. AUGUST         1200
# 09. SEPTEMBER     28742
# 10. OCTOBER       40539
# 11. NOVEMBER      11273
# 12. DECEMBER       5473
# 99. MISSING      128257

# IMD5024_2    >>>         DATE ELECTION SCHEDULED - DAY
# 1                                   3400
# 2                                   5710
# 3                                   3500
# 4                                   4637
# 5                                   6686
# 6                                   4497
# 7                                   9918
# 8                                   2591
# 9                                  12234
# 10                                  6342
# 11                                  3153
# 12                                  5919
# 13                                  1200
# 14                                  5640
# 15                                  3458
# 17                                  3874
# 18                                  1283
# 19                                 10284
# 20                                  1224
# 21                                  9004
# 22                                 10801
# 23                                  6381
# 24                                  1435
# 25                                  5780
# 26                                  1374
# 27                                  4890
# 28                                  1000
# 29                                  1967
# 96. [SEE ELECTION STUDY NOTES]      1653
# 99. MISSING                       141248

# IMD5024_3    >>>         DATE ELECTION SCHEDULED - YEAR
# 2005               4414
# 2006               9358
# 2007              14856
# 2008              10230
# 2009              23322
# 2010               6573
# 2011              18721
# 2012              22635
# 2013              13969
# 2014              11297
# 2015              12544
# 2016               4930
# 2017                999
# 9999. MISSING    127235

# IMD5025_1    >>>         DATE ELECTION HELD - MONTH
# 01. JANUARY                        6144
# 02. FEBRUARY                       4654
# 03. MARCH                         26287
# 04. APRIL                         24596
# 05. MAY                           35097
# 06. JUNE                          21873
# 07. JULY                          23825
# 09. SEPTEMBER                     47299
# 10. OCTOBER                       50837
# 11. NOVEMBER                      25492
# 12. DECEMBER                      13750
# 96. [SEE ELECTION STUDY NOTES]     1229

# IMD5025_2    >>>         DATE ELECTION HELD - DAY
# 1                                  8319
# 2                                 15649
# 3                                  8005
# 4                                  7890
# 5                                 10079
# 6                                 14217
# 7                                 11118
# 8                                  3898
# 9                                 18433
# 10                                18709
# 11                                 6178
# 12                                12962
# 13                                 9663
# 14                                10828
# 15                                 6189
# 16                                 1196
# 17                                10030
# 18                                 5526
# 19                                 9049
# 20                                10358
# 21                                11985
# 22                                14810
# 23                                 9375
# 24                                 6356
# 25                                10026
# 26                                 3122
# 27                                 8650
# 28                                10021
# 29                                 4612
# 96. [SEE ELECTION STUDY NOTES]     3830

# IMD5025_3    >>>         DATE ELECTION HELD - YEAR
# 1996    16677
# 1997    11848
# 1998    10951
# 1999    10833
# 2000     7598
# 2001    12575
# 2002    16730
# 2003     9488
# 2004    17214
# 2005    14409
# 2006    13170
# 2007    18976
# 2008    15890
# 2009    14771
# 2010     9418
# 2011    21507
# 2012    16060
# 2013    15855
# 2014     9639
# 2015    13552
# 2016     3922

# IMD5026_1    >>>         NUMBER OF LEGISLATIVE CHAMBERS
# 1. UNICAMERAL LEGISLATURE (ONE LEGISLATIVE CHAMBER)     99110
# 2. BICAMERAL LEGISLATURE (TWO LEGISLATIVE CHAMBERS)    181973

# IMD5026_2    >>>         NUMBER OF ELECTED LEGISLATIVE CHAMBERS  
# 0. UNICAMERAL LEGISLATURE                                                  99110
# 1. BICAMERAL LEGISLATURE: INDIRECTLY ELECTED OR APPOINTED                  67774
# 2. BICAMERAL LEGISLATURE: FULLY DIRECTLY ELECTED                           94497
# 6. BICAMERAL LEGISLATURE: PARTIALLY DIRECTLY ELECTED [SEE POLITY NOTES]    19702
    
# IMD5027      >>>         SIZE OF THE LOWER HOUSE
# Total number of seats in the lower house of the legislature during the election year.
 
# 60     3071
# 63     7536
# 70     1044
# 75     2000
# 81      967
#        ... 
# 631    1889
# 646     860
# 650    1567
# 659    2897
# 669    2019

# IMD5028      >>>         SIZE OF CABINET BEFORE
# The size of the cabinet before the election.
#  | a) Parliamentary and Semi-Presidential Regimes:
#  |    Cabinet size is defined by the total number of ministers
#  |    (persons, not posts) in a defined government. Ministers are
#  |    considered members of a cabinet when they exercise voting
#  |    rights. This number includes both ministers with and without
#  |    portfolio, but excludes deputy ministers, undersecretaries,
#  |    parliamentary secretaries, ministerial alternates, given that
#  |    in the majority of cases, they do not exercise full voting
#  |    rights.
#  | b) Presidential Regimes:
#  |    Cabinet size is defined by the total number of ministers or
#  |    secretaries who head a ministry.
# 7                8973
# 8                1479
# 11               1403
# 12               5626
# 13               3514
#                 ....
# 37               2000
# 38               4458
# 39               9012
# 40               1200
# 45               3731
# 46               3845
# 999. MISSING    73925

# IMD5029_A    >>>         NUMBER OF PORFOLIOS BEFORE - PARTY A
#  Number of cabinet posts (portfolios) held by PARTY
#  [A/B/C/D/E/F/G/H/I] before the election.
# 0               72876
# 1                6853
# 2                6179
# 3                1200
#             ...
# 26               1086
# 27               5604
# 28               1200
# 29               2000
# 34               1300
# 38               3458
# 39               5876
# 46               1823
# 999. MISSING    75915

# IMD5029_B    >>>         NUMBER OF PORFOLIOS BEFORE - PARTY B
# IMD5029_C    >>>         NUMBER OF PORFOLIOS BEFORE - PARTY C
# IMD5029_D    >>>         NUMBER OF PORFOLIOS BEFORE - PARTY D
# IMD5029_E    >>>         NUMBER OF PORFOLIOS BEFORE - PARTY E
# IMD5029_F    >>>         NUMBER OF PORFOLIOS BEFORE - PARTY F
# IMD5029_G    >>>         NUMBER OF PORFOLIOS BEFORE - PARTY G
# IMD5029_H    >>>         NUMBER OF PORFOLIOS BEFORE - PARTY H
# IMD5029_I    >>>         NUMBER OF PORFOLIOS BEFORE - PARTY I
# 0                65896
# 2                 2225
# 4                 1200
# 999. MISSING    211762

# IMD5030      >>>         SIZE OF CABINET AFTER
# The size of the cabinet after the election.
# 7                8973
# 9                1479
# 12               6579
#             ...
# 39               3458
# 45               1905
# 47               3845
# 999. MISSING    71899

# IMD5031_A    >>>         NUMBER OF PORFOLIOS AFTER - PARTY A
#  Number of cabinet posts (portfolios) held by PARTY
#  [A/B/C/D/E/F/G/H/I] after the election.
# 0               31050
# 1                6250
# 2                4582
# 3                2400
#                 .
# 35               1300
# 36               1826
# 38               4495
# 39               3458
# 47               1823
# 999. MISSING    71899

# IMD5031_B    >>>         NUMBER OF PORFOLIOS AFTER - PARTY B
# IMD5031_C    >>>         NUMBER OF PORFOLIOS AFTER - PARTY C
# IMD5031_D    >>>         NUMBER OF PORFOLIOS AFTER - PARTY D
# IMD5031_E    >>>         NUMBER OF PORFOLIOS AFTER - PARTY E
# IMD5031_F    >>>         NUMBER OF PORFOLIOS AFTER - PARTY F
# IMD5031_G    >>>         NUMBER OF PORFOLIOS AFTER - PARTY G
# IMD5031_H    >>>         NUMBER OF PORFOLIOS AFTER - PARTY H
# IMD5031_I    >>>         NUMBER OF PORFOLIOS AFTER - PARTY I
# 0                68338
# 1                 6775
# 999. MISSING    205970


# IMD5049      >>>         AGE OF CURRENT REGIME
#  The number of years since the most recent regime change (defined
#  by a three-point change in the POLITY score over a period of three
#  years or less) or the end of transition period defined by the lack
#  of stable political institutions (denoted by a standardized
#  authority score) [Variable "Durable" from Polity IV Project
#  Dataset Users' Manual].
## LOOK UP POLITY DATASET
# 0               12666
# 3                2995
# 4                2696
# 5                5063
# 6                7538
#                 ...  
# 187              1534
# 195              1066
# 199              2102
# 203              1929
# 999. MISSING    11651

In [ ]:
# IMD5050_1    >>>         FREEDOM HOUSE RATING - TIME T
#  These variables report Freedom House's rating of freedom in a
#  country at three time periods: the election year (time T), one
#  year before election (T-1), and two years before election (T-2).
#  Each country and territory is assigned a numerical rating, on a
#  scale of 1 to 7. A rating of 1 indicates the highest degree of
#  freedom and 7 the least amount of freedom.
#  CSES reports average of the "Political Rights" and "Civil
#  Liberties" scores.
# 1.0    151817
# 1.5     44143
# 2.0     35451
# 2.5     18190
# 3.0      7022
# 3.5      9010
# 4.0      4374
# 4.5      3842
# 5.0      1748
# 5.5      3486
# 6.0      1000
# 6.5      1000
# IMD5050_2    >>>         FREEDOM HOUSE RATING - TIME T-1 YEAR
# 1.0    145392
# 1.5     50283
# 2.0     31676
# 2.5     14477
# 3.0     12726
# 3.5     12585
# 4.0      3098
# 4.5      5350
# 5.0      1496
# 5.5      2000
# 6.0      1000
# 6.5      1000
# IMD5050_3    >>>         FREEDOM HOUSE RATING - TIME T-2 YEARS
# 1.0    143070
# 1.5     48056
# 2.0     35407
# 2.5     17285
# 3.0     11736
# 3.5     11276
# 4.0      5037
# 4.5      3720
# 5.0      1496
# 5.5      2000
# 6.0      1000
# 6.5      1000

In [ ]:
# IMD5051_1    >>>         DEMOCRACY-AUTOCRACY - POLITY IV RATING - TIME T
#  These variables report POLITY IV ratings of institutionalized
#  democracy versus autocracy in a country, at three time periods:
#  the election year (time T), one year before election (T-1), and
#  two years before election (T-2).

#  CSES reports the original variable POLITY - Combined Polity Score.
#  The variable is constructed by subtracting the autocracy score
#  from the democracy score; the resulting scale ranges from +10
#  (strongly democratic) to -10 (strongly autocratic).

# -88. Transition Periods      1102
# -7                           2000
# -6                           2000
# -1                           1990
# 3                            2928
# 6                            5277
# 7                            2648
# 8                           40215
# 9                           40074
# 10. Democratic             169628
# 99. MISSING                 13221

# IMD5051_2    >>>         DEMOCRACY-AUTOCRACY - POLITY IV RATING - TIME T-1 YEAR
# -88. Transition Periods      1118
# -7                           2000
# -6                           2000
# -5                           1990
# 1                            1102
# 3                            3590
# 4                            3533
# 5                            1175
# 6                            3262
# 7                            4573
# 8                           40073
# 9                           35209
# 10. Democratic             169807
# 99. MISSING                 11651

# IMD5051_3    >>>         DEMOCRACY-AUTOCRACY - POLITY IV RATING - TIME T-2 YEARS
# -7                  2000
# -6                  2000
# 1                   2220
# 3                   3590
# 4                   3533
# 5                   1175
# 6                   3262
# 7                   4573
# 8                  40073
# 9                  37199
# 10. Democratic    169807
# 99. MISSING        11651

In [ ]:
# IMD5052_1    >>>         GDP GROWTH ANNUAL % (WORLD BANK) - TIME T
# -8.5                                                   1403
# -6.97                                                  1029
# -6.54                                                  2400
# -6.49                                                  1385
# -5.88                                                  1000
#                                                        ... 
# 9.56                                                   1000
# 10.0                                                   1748
# 10.2                                                   1000
# 10.43                                                  1913
# 99.00. MISSING/NOT AVAILABLE AT TIME OF PUBLICATION    5045

# IMD5052_2    >>>         GDP GROWTH ANNUAL % (WORLD BANK) - TIME T-1
# -17.95                                                 1005
# -6.97                                                  1029
# -6.2                                                   1203
# -5.3                                                   1842
# -3.54                                                  2153
#                                                        ... 
# 8.82                                                   1570
# 9.16                                                   1109
# 9.43                                                   1403
# 11.31                                                  1100
# 99.00. MISSING/NOT AVAILABLE AT TIME OF PUBLICATION    5045

# IMD5052_3    >>>         GDP GROWTH ANNUAL % (WORLD BANK) - TIME T-2
# -14.1                                                  1000
# -10.0                                                  1148
# -8.41                                                  1482
# -8.35                                                  1298
# -6.97                                                  1029
#                                                        ... 
# 7.43                                                   3764
# 7.48                                                   1595
# 9.46                                                   2367
# 10.0                                                   1000
# 99.00. MISSING/NOT AVAILABLE AT TIME OF PUBLICATION    6054

# IMD5053_1    >>>         GDP PER CAPITA, PPP (WORLD BANK) - TIME T
# 2111.76                                                    2000
# 2843.77                                                    1200
# 3415.55                                                    1148
# 3560.5                                                     1200
# 4013.77                                                    1200
#                                                            ... 
# 53142.89                                                   1929
# 58938.77                                                    832
# 67468.07                                                   3953
# 102832.26                                                  1727
# 999999.00. MISSING/NOT AVAILABLE AT TIME OF PUBLICATION    5045

# IMD5053_2    >>>         GDP PER CAPITA, PPP (WORLD BANK) - TIME T-1
# 2074.84                                                    2000
# 2718.99                                                    1200
# 3364.21                                                    1200
# 3412.31                                                    1148
# 3736.36                                                    1200
#                                                            ... 
# 51755.21                                                   1929
# 60283.25                                                    832
# 67435.95                                                   3953
# 101563.7                                                   1727
# 999999.00. MISSING/NOT AVAILABLE AT TIME OF PUBLICATION    5045

# IMD5053_3    >>>         GDP PER CAPITA, PPP (WORLD BANK) - TIME T-2
# 1910.77                                                    2000
# 2622.73                                                    1200
# 3382.45                                                    1200
# 3426.85                                                    1148
# 3566.98                                                    1200
#                                                            ... 
# 49854.52                                                   1929
# 57134.08                                                    832
# 62080.98                                                   3953
# 100575.1                                                   1727
# 999999.00. MISSING/NOT AVAILABLE AT TIME OF PUBLICATION    5045

In [ ]:
# IMD5054_1    >>>         UNEMPLOYMENT, TOTAL (WORLD BANK) - TIME T
#   These variables report World Bank estimates of the unemployment
#   rate (% of total labor force), at three time periods: the
#   election year (time T), one year before election  (T-1), and two
#   years before election (T-2).

#   Unemployment is the share of the labor force without work but
#   available for and seeking employment. Definitions of labor force
#   and unemployment may differ by country.
# 0.7                                                     1500
# 0.8                                                     1000
# 1.2                                                     1990
# 2.18                                                    1631
# 2.3                                                     1595
#                                                         ... 
# 23.9                                                    1568
# 24.2                                                    1029
# 24.9                                                    1008
# 25.1                                                    1300
# 999.00. MISSING/NOT AVAILABLE AT TIME OF PUBLICATION    5045

# IMD5054_2    >>>         UNEMPLOYMENT, TOTAL (WORLD BANK) - TIME T-1
# 1.0                                                     2500
# 1.2                                                     1990
# 2.12                                                    1574
# 2.22                                                    1000
# 2.39                                                    1081
#                                                         ... 
# 22.9                                                    1200
# 23.0                                                    1568
# 24.6                                                    1300
# 26.5                                                    1008
# 999.00. MISSING/NOT AVAILABLE AT TIME OF PUBLICATION    5045

# IMD5054_3    >>>         UNEMPLOYMENT, TOTAL (WORLD BANK) - TIME T-2
# 1.2                                                     1000
# 1.3                                                     1990
# 1.5                                                     1500
# 1.87                                                    1446
# 2.3                                                     1385
#                                                         ... 
# 23.0                                                    1200
# 24.21                                                   1212
# 25.0                                                    1300
# 27.5                                                    1008
# 999.00. MISSING/NOT AVAILABLE AT TIME OF PUBLICATION    5045

In [440]:
# IMD5055_1    >>>         HUMAN DEVELOPMENT INDEX (UNPD)  - TIME T
# 0.54                                                     1200
# 0.61                                                     1200
# 0.62                                                     2000
#                             ...
# 0.9                                                     14228
# 0.91                                                    22358
# 0.92                                                     9481
# 0.93                                                     7838
# 0.94                                                     3509
# 999.00. MISSING/NOT AVAILABLE AT TIME OF PUBLICATION     8776
#  These variables report UNDP Human Development Index (HDI)
#  at three time periods: the election year (time T), one year
#  before election (T-1), and two years before election (T-2).

#  Definition: The human development index (HDI) is a composite index
#  that measures the average achievements in a country in three basic
#  dimensions of human development: a long and healthy life; access
#  to knowledge; and a decent standard of living. These basic
#  dimensions are measured by life expectancy at birth, adult
#  literacy and combined gross enrollment in primary, secondary and
#  tertiary level education, and gross domestic product (GDP) per
#  capita in Purchasing Power Parity US dollars (PPP US$),
#  respectively.

# IMD5055_2    >>>         HUMAN DEVELOPMENT INDEX (UNPD)  - TIME T-1
# IMD5055_3    >>>         HUMAN DEVELOPMENT INDEX (UNPD)  - TIME T-2

# IMD5056_1    >>>         INFLATION, GDP DEFLATOR (ANNUAL %) (WORLD BANK) - TIME T
# -5.55                                                     1782
# -3.59                                                      582
# -3.39                                                      674
# -2.32                                                     1005
# -1.1                                                      1977
#                                                           ... 
# 37.7                                                      1748
# 44.0                                                      1175
# 72.39                                                     1842
# 79.53                                                     1000
# 99999.00. MISSING/NOT AVAILABLE AT TIME OF PUBLICATION    5045

# IMD5056_2    >>>         INFLATION, GDP DEFLATOR (ANNUAL %) (WORLD BANK) - TIME T-1
# -6.01                                                      582
# -4.1                                                       674
# -2.22                                                     1853
# -1.83                                                     1008
# -1.62                                                     1977
#                                                           ... 
# 40.72                                                     1406
# 43.93                                                     1175
# 72.39                                                     1748
# 185.29                                                    1000
# 99999.00. MISSING/NOT AVAILABLE AT TIME OF PUBLICATION    5045

# IMD5056_3    >>>         INFLATION, GDP DEFLATOR (ANNUAL %) (WORLD BANK) - TIME T-2
# -9.69                                                     1004
# -4.64                                                     1853
# -3.4                                                       582
# -2.58                                                     1081
# -2.35                                                     1008
#                                                           ... 
# 38.46                                                     2033
# 66.15                                                     1148
# 139.02                                                    1175
# 316.79                                                    1000
# 99999.00. MISSING/NOT AVAILABLE AT TIME OF PUBLICATION    7880

# IMD5057_1    >>>         POPULATION, TOTAL (WORLD BANK) - TIME T
# 277381                                                            1631
# 289521                                                            1446
# 311566                                                            1595
# 319062                                                            1385
# 323002                                                            1479
#                                                                   ... 
# 269394000                                                         1534
# 292805298                                                         1066
# 304375008                                                         2102
# 313873696                                                         1929
# 999999999. MISSING AND/OR NOT AVAILABLE AT TIME OF PUBLICATION    5045

# IMD5057_2    >>>         POPULATION, TOTAL (WORLD BANK) - TIME T-1
# 274047                                                            1631
# 287523                                                            1446
# 303782                                                            1595
# 317414                                                            1385
# 320716                                                            1479
#                                                                   ... 
# 266278000                                                         1534
# 290107933                                                         1066
# 301580000                                                         2102
# 311582560                                                         1929
# 999999999. MISSING AND/OR NOT AVAILABLE AT TIME OF PUBLICATION    6871

# IMD5057_3    >>>         POPULATION, TOTAL (WORLD BANK) - TIME T-2
# 271128                                                            1631
# 284968                                                            1446
# 296750                                                            1595
# 311566                                                            1385
# 319014                                                            1479
#                                                                   ... 
# 263126000                                                         1534
# 287625193                                                         1066
# 298592992                                                         2102
# 309326304                                                         1929
# 999999999. MISSING AND/OR NOT AVAILABLE AT TIME OF PUBLICATION    6871

# IMD5058_1    >>>         EFFECTIVE NUMBER OF ELECTORAL PARTIES
# 1.32                                                    1000
# 2.16                                                    2102
# 2.18                                                    2600
# 2.19                                                    1929
# 2.22                                                    1200
#                                                        ...  
# 11.21                                                   2000
# 11.24                                                   1842
# 14.1                                                    3136
# 997.00. PRESIDENTIAL ELECTION ONLY - NOT CALCULATED    14575
# 999.00. MISSING                                         9684

# IMD5058_2    >>>         CORRECTED EFFECTIVE NUMBER OF ELECTORAL PARTIES
# 2.15                                                     2102
# 2.19                                                     1929
# 2.22                                                     1200
# 2.27                                                     1300
# 2.29                                                     1905
#                                                         ...  
# 11.21                                                    2000
# 13.68                                                    3136
# 150.96                                                   1000
# 997.00. PRESIDENTIAL ELECTION ONLY - NOT CALCULATED      3126
# 999.00. MISSING                                        125362

# IMD5059_1    >>>         EFFECTIVE NUMBER OF PARLIAMENTARY PARTIES
# 1.16                                                    1000
# 1.75                                                    1905
# 1.94                                                    2102
# 1.99                                                    1929
# 2.0                                                     4426
#                                                        ...  
# 9.73                                                    1000
# 10.36                                                   2000
# 13.33                                                   3136
# 997.00. PRESIDENTIAL ELECTION ONLY - NOT CALCULATED    14575
# 999.00. MISSING                                        10892

# IMD5059_2    >>>         CORRECTED EFFECTIVE NUMBER OF PARLIAMENTARY PARTIES
# 1.75                                                     1905
# 1.94                                                     2102
# 1.99                                                     1929
# 2.0                                                      1826
# 2.11                                                     2283
#                                                         ...  
# 10.35                                                    2000
# 13.17                                                    3136
# 121.76                                                   1000
# 997.00. PRESIDENTIAL ELECTION ONLY - NOT CALCULATED      3126
# 999.00. MISSING                                        125362

# CSES_int["IMD5059_2"].cat.remove_unused_categories().value_counts().sort_index()

In [441]:
gc.collect()
["IMD1003"]

140

In [461]:
max_dist_values_ES = CSES_int.groupby(CSES_int["IMD1003"]).apply(lambda x: x.apply(lambda x: len(x.unique()))).max()

In [495]:
# CSES_int.columns[CSES_int.apply(lambda x: len(x.unique()))==1]
empty_columns = ['IMD1016_3']
dataset_columns = ['IMD1001', 'IMD1002_VER', 'IMD1002_DOI', ]


In [496]:
party_level = [x for x in max_dist_values_ES[max_dist_values_ES==1].index if (x not in empty_columns and x not in dataset_columns) and re.match("IMD\d+_[A-I]$",x)]
# grabs some _D "day variables too!"

party_level.remove("IMD1011_D")
party_level.remove("IMD1012_D")

In [499]:
country_level = [x for x in max_dist_values_ES[max_dist_values_ES==1].index if (x not in empty_columns) and (x not in dataset_columns) and (x not in party_level) ]

pd.Series(headerINT).loc[country_level]

IMD1003            ID VARIABLE - ELECTION STUDY (NUMERIC POLITY)
IMD1004         ID VARIABLE - ELECTION STUDY (ALPHABETIC POLITY)
IMD1006                          ID COMPONENT - POLITY CSES CODE
IMD1006_UN                         ID COMPONENT - POLITY UN CODE
IMD1006_NAM                           ID COMPONENT - POLITY NAME
                                     ...                        
IMD5057_3              POPULATION, TOTAL (WORLD BANK) - TIME T-2
IMD5058_1                  EFFECTIVE NUMBER OF ELECTORAL PARTIES
IMD5058_2        CORRECTED EFFECTIVE NUMBER OF ELECTORAL PARTIES
IMD5059_1              EFFECTIVE NUMBER OF PARLIAMENTARY PARTIES
IMD5059_2      CORRECTED EFFECTIVE NUMBER OF PARLIAMENTARY PA...
Length: 86, dtype: object

In [ ]:
# empty_columns
# dataset_columns (maybe add module 1/2/3/4 cols?)
# party_level (party-country-year)
# country-level (election_study-country-year)

In [508]:
df = pd.DataFrame(pd.Series(headerINT), columns=["description"])
df["variable type"] = np.nan
for el in empty_columns:
    df.loc[el,"variable type"] = "empty"
for el in dataset_columns:
    df.loc[el,"variable type"] = "dataset"
for el in party_level:
    df.loc[el,"variable type"] = "party"
for el in country_level:
    df.loc[el,"variable type"] = "country"
    

In [512]:
df["variable type"][df["variable type"].isnull()] = "int only"

In [514]:
df.to_pickle(dataset_folder + "CSES"+os.sep+"Integration"+os.sep+"cses_variable_types.pkl")

In [515]:
gc.collect()

5097